# 4th_2006

## Raw Data

In [1]:
# =============================================================================
# 4대 지방선거 데이터 전처리 함수들
# =============================================================================

import pandas as pd
import re
import requests
import tempfile
import os

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_4th_governor_election(
    file_path_or_url,
    header_rows=(4, 6),
    filter_column=None,
    filter_value=None
):
    """
    선거 데이터 전처리 함수 (17대 대선 코드 패턴 적용)
    """
    # URL인 경우 처리
    if file_path_or_url.startswith(('http://', 'https://')):
        # GitHub URL 변환
        if 'github.com' in file_path_or_url:
            file_path_or_url = convert_github_url_to_raw(file_path_or_url)

        # 임시 파일로 다운로드
        with tempfile.NamedTemporaryFile(delete=False, suffix=".xls") as tmp:
            response = requests.get(file_path_or_url)
            response.raise_for_status()
            tmp.write(response.content)
            tmp_path = tmp.name

        try:
            df_raw = pd.read_excel(tmp_path, header=None)
        finally:
            os.remove(tmp_path)
    else:
        # 로컬 파일인 경우
        df_raw = pd.read_excel(file_path_or_url, header=None)

    # 헤더 처리
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = [row_data[col_idx] for row_data in header_rows_data if row_data[col_idx] != '']
        new_columns.append('_'.join(parts) if parts else f'column_{col_idx}')

    # 데이터 추출
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 필터링
    if filter_column and filter_value:
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")
            filtered_df = df[df[matching_col] == filter_value].copy()
            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")

    return df


def process_4th_governor_election_batch(user, repo, folder):
    """
    4대 지방선거 시도지사 선거 데이터 일괄 처리 함수

    Parameters:
    -----------
    user : str
        GitHub 사용자명
    repo : str
        GitHub 저장소명
    folder : str
        데이터가 있는 폴더 경로

    Returns:
    --------
    pd.DataFrame
        병합된 선거 데이터

    Example:
    --------
    df4th = process_4th_governor_election_batch(
        user="checkdroid",
        repo="korean-elections",
        folder="original/Local_Elections_Governor/4th_2006/11_서울"
    )
    """
    print(f"4대 지방선거 데이터 일괄 처리 시작")
    print(f"저장소: {user}/{repo}")
    print(f"폴더: {folder}")
    print("-" * 60)

    # ================================
    # 1. GitHub API로 파일 목록 가져오기
    # ================================
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        files = response.json()
        print(f"폴더에서 {len(files)}개 파일 발견")
    except Exception as e:
        print(f"API 요청 실패: {e}")
        return None

    # ================================
    # 2. 각 .xls 파일 처리
    # ================================
    processed_dfs = []
    success_count = 0
    failed_files = []

    for file_info in files:
        file_name = file_info["name"]

        # .xls 파일만 처리
        if not file_name.endswith(".xls"):
            continue

        # 파일명에서 지역명 추출 (예: "1101_종로구_서울특별시.xls" → "종로구")
        match = re.match(r"^\d+_([^_]+)_[^_]+\.xls$", file_name)
        if not match:
            print(f"파일명 패턴 불일치, 무시: {file_name}")
            continue

        region = match.group(1)
        print(f"\n처리 중: {file_name} (지역: {region})")

        # raw URL 생성
        raw_url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{folder}/{file_name}"

        try:
            # 임시 파일로 다운로드 후 처리
            with tempfile.NamedTemporaryFile(delete=False, suffix=".xls") as tmp:
                response = requests.get(raw_url)
                response.raise_for_status()
                tmp.write(response.content)
                tmp_path = tmp.name

            # 개별 파일 전처리
            df_region = process_4th_governor_election(
                tmp_path,
                header_rows=(4, 6),
                filter_column='읍면동명',
                filter_value='합계'
            )

            # 지역명 추가
            if df_region is not None and not df_region.empty:
                df_region.insert(0, '구시군명', region)
                processed_dfs.append(df_region)
                success_count += 1
                print(f"  ✓ 완료: {df_region.shape[0]}행 추가")
            else:
                print(f"  ⚠ 경고: 데이터가 비어있거나 필터링 결과 없음")
                failed_files.append(file_name)

            # 임시 파일 삭제
            os.remove(tmp_path)

        except Exception as e:
            print(f"  ✗ 오류 발생: {type(e).__name__}: {str(e)}")
            failed_files.append(file_name)
            # 임시 파일이 있다면 삭제
            if 'tmp_path' in locals() and os.path.exists(tmp_path):
                os.remove(tmp_path)
            continue

    # ================================
    # 3. 모든 DataFrame 병합
    # ================================
    print("\n" + "=" * 60)
    if processed_dfs:
        df_combined = pd.concat(processed_dfs, ignore_index=True)
        print(f"일괄 처리 완료:")
        print(f"  - 성공: {success_count}개 파일")
        print(f"  - 실패: {len(failed_files)}개 파일")
        if failed_files:
            print(f"  - 실패 파일: {', '.join(failed_files[:5])}")
            if len(failed_files) > 5:
                print(f"    ... 외 {len(failed_files)-5}개")
        print(f"  - 최종 크기: {df_combined.shape[0]}행 x {df_combined.shape[1]}열")
        return df_combined
    else:
        print("처리된 파일이 없습니다.")
        return None

## Seoul


In [2]:
seoul_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/11_서울"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/11_서울
------------------------------------------------------------
폴더에서 25개 파일 발견

처리 중: 1101_종로구_서울특별시.xls (지역: 종로구)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 20
데이터 행 수: 29

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 29행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 1102_중구_서울특별시.xls (지역: 중구)
WARNING *** file size (14357) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 20
데이터 행 수: 24

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 24행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 1103_용산구_서울특별시.xls (지역: 용산구)
WARNING *** file size (16823) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 20
데이터 행 수: 30

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 30행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 1104_성동구_서울특별시.xls (지역: 성동구)
WARNING *** file size (16956) n

In [3]:
seoul_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n강금실,한나라당\n오세훈,민주당\n박주선,민주노동당\n김종철,...,시민당\n이귀선,한미준\n이태희,무소속\n백승원,column_13,계,무효\n투표수,기권수,column_17,column_18,column_19
0,종로구,NaN,합계,"134,603",NaN,"71,240","19,369","41,992","6,460","1,945",...,110,89,265,NaN,"70,466",774,"63,363",NaN,NaN,0.0
1,중구,NaN,합계,"107,164",NaN,"55,369","15,382","32,708","4,609","1,455",...,193,113,212,NaN,"54,858",511,"51,795",NaN,NaN,0.0
2,용산구,NaN,합계,"187,461",NaN,"95,809","26,087","58,427","6,970","2,632",...,152,124,319,NaN,"95,029",780,"91,652",NaN,NaN,0.0
3,성동구,NaN,합계,"266,956",NaN,"131,927","35,842","77,631","11,971","3,975",...,165,153,488,NaN,"130,686","1,241","135,029",NaN,NaN,0.0
4,광진구,NaN,합계,"293,308",NaN,"141,079","39,716","83,209","11,693","4,070",...,228,121,443,NaN,"140,012","1,067","152,229",NaN,NaN,0.0
5,동대문구,NaN,합계,"305,097",NaN,"153,571","40,952","92,583","12,817","4,344",...,231,171,509,NaN,"152,089","1,482","151,526",NaN,NaN,0.0
6,중랑구,NaN,합계,"332,783",NaN,"155,626","41,950","94,016","12,084","4,711",...,236,177,656,NaN,"154,367","1,259","177,157",NaN,NaN,0.0
7,성북구,NaN,합계,"369,617",NaN,"183,552","52,218","107,844","14,592","5,856",...,191,239,620,NaN,"182,138","1,414","186,065",NaN,NaN,0.0
8,강북구,NaN,합계,"280,448",NaN,"133,939","37,446","75,788","13,997","4,146",...,207,187,577,NaN,"132,776","1,163","146,509",NaN,NaN,0.0
9,도봉구,NaN,합계,"291,852",NaN,"147,638","41,460","88,101","11,290","4,542",...,113,108,450,NaN,"146,624","1,014","144,214",NaN,NaN,0.0


In [4]:
seoul_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n강금실',
 '한나라당\n오세훈',
 '민주당\n박주선',
 '민주노동당\n김종철',
 '국민중심당\n임웅균',
 '시민당\n이귀선',
 '한미준\n이태희',
 '무소속\n백승원',
 'column_13',
 '계',
 '무효\n투표수',
 '기권수',
 'column_17',
 'column_18',
 'column_19']

In [5]:
rename_seoul = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n강금실': '득표수_1_열린우리당_강금실',
    '한나라당\n오세훈': '득표수_2_한나라당_오세훈',
    '민주당\n박주선': '득표수_3_민주당_박주선',
    '민주노동당\n김종철': '득표수_4_민주노동당_김종철',
    '국민중심당\n임웅균': '득표수_5_국민중심당_임웅균',
    '시민당\n이귀선': '득표수_6_시민당_이귀선',
    '한미준\n이태희': '득표수_7_한국의미래를준비하는당_이태희',
    '무소속\n백승원': '득표수_8_무소속_백승원',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [6]:
seoul_4th = seoul_4th.rename(columns=rename_seoul).drop(columns=['column_0', '읍면동명', 'column_3', 'column_13', 'column_17', 'column_18', 'column_19'])
seoul_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_강금실,득표수_2_한나라당_오세훈,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수
0,종로구,"134,603","71,240","19,369","41,992","6,460","1,945",236,110,89,265,"70,466",774,"63,363"
1,중구,"107,164","55,369","15,382","32,708","4,609","1,455",186,193,113,212,"54,858",511,"51,795"
2,용산구,"187,461","95,809","26,087","58,427","6,970","2,632",318,152,124,319,"95,029",780,"91,652"
3,성동구,"266,956","131,927","35,842","77,631","11,971","3,975",461,165,153,488,"130,686","1,241","135,029"
4,광진구,"293,308","141,079","39,716","83,209","11,693","4,070",532,228,121,443,"140,012","1,067","152,229"
5,동대문구,"305,097","153,571","40,952","92,583","12,817","4,344",482,231,171,509,"152,089","1,482","151,526"
6,중랑구,"332,783","155,626","41,950","94,016","12,084","4,711",537,236,177,656,"154,367","1,259","177,157"
7,성북구,"369,617","183,552","52,218","107,844","14,592","5,856",578,191,239,620,"182,138","1,414","186,065"
8,강북구,"280,448","133,939","37,446","75,788","13,997","4,146",428,207,187,577,"132,776","1,163","146,509"
9,도봉구,"291,852","147,638","41,460","88,101","11,290","4,542",560,113,108,450,"146,624","1,014","144,214"


In [7]:
seoul_4th = seoul_4th.assign(
    시도='서울특별시'
)[['시도'] + seoul_4th.columns.tolist()]

In [8]:
seoul_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_강금실,득표수_2_한나라당_오세훈,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수
0,서울특별시,종로구,"134,603","71,240","19,369","41,992","6,460","1,945",236,110,89,265,"70,466",774,"63,363"
1,서울특별시,중구,"107,164","55,369","15,382","32,708","4,609","1,455",186,193,113,212,"54,858",511,"51,795"
2,서울특별시,용산구,"187,461","95,809","26,087","58,427","6,970","2,632",318,152,124,319,"95,029",780,"91,652"
3,서울특별시,성동구,"266,956","131,927","35,842","77,631","11,971","3,975",461,165,153,488,"130,686","1,241","135,029"
4,서울특별시,광진구,"293,308","141,079","39,716","83,209","11,693","4,070",532,228,121,443,"140,012","1,067","152,229"


In [9]:
seoul_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   시도                     25 non-null     object
 1   구시군                    25 non-null     object
 2   선거인수                   25 non-null     object
 3   투표수                    25 non-null     object
 4   득표수_1_열린우리당_강금실        25 non-null     object
 5   득표수_2_한나라당_오세훈         25 non-null     object
 6   득표수_3_민주당_박주선          25 non-null     object
 7   득표수_4_민주노동당_김종철        25 non-null     object
 8   득표수_5_국민중심당_임웅균        25 non-null     object
 9   득표수_6_시민당_이귀선          25 non-null     object
 10  득표수_7_한국의미래를준비하는당_이태희  25 non-null     object
 11  득표수_8_무소속_백승원          25 non-null     object
 12  득표수_계                  25 non-null     object
 13  무효투표수                  25 non-null     object
 14  기권수                    25 non-null     object
dtypes: object(15)
memory usag

In [10]:
seoul_4th = seoul_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [11]:
# 수치형 열만 합계 구하기
summary_row = seoul_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '서울특별시')

# summary_row를 맨 위에 붙이기
seoul_4th_with_total = pd.concat([summary_row, seoul_4th], ignore_index=True)

In [12]:
seoul_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_강금실,득표수_2_한나라당_오세훈,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수
0,서울특별시,합계,7983648,3977842,1077890,2409760,304565,117421,14111,4790,4481,13808,3946826,31016,4005806
1,서울특별시,종로구,134603,71240,19369,41992,6460,1945,236,110,89,265,70466,774,63363
2,서울특별시,중구,107164,55369,15382,32708,4609,1455,186,193,113,212,54858,511,51795
3,서울특별시,용산구,187461,95809,26087,58427,6970,2632,318,152,124,319,95029,780,91652
4,서울특별시,성동구,266956,131927,35842,77631,11971,3975,461,165,153,488,130686,1241,135029
5,서울특별시,광진구,293308,141079,39716,83209,11693,4070,532,228,121,443,140012,1067,152229
6,서울특별시,동대문구,305097,153571,40952,92583,12817,4344,482,231,171,509,152089,1482,151526
7,서울특별시,중랑구,332783,155626,41950,94016,12084,4711,537,236,177,656,154367,1259,177157
8,서울특별시,성북구,369617,183552,52218,107844,14592,5856,578,191,239,620,182138,1414,186065
9,서울특별시,강북구,280448,133939,37446,75788,13997,4146,428,207,187,577,132776,1163,146509


In [13]:
seoul_4th_with_total.to_csv("temp1_governor_seoul_4.csv", index=False, encoding="utf-8-sig")


## Busan


In [14]:
busan_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/26_부산"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/26_부산
------------------------------------------------------------
폴더에서 16개 파일 발견

처리 중: 2601_중구_부산광역시.xls (지역: 중구)
WARNING *** file size (117881) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 12

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 12행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2602_서구_부산광역시.xls (지역: 서구)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 24

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 24행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2603_동구_부산광역시.xls (지역: 동구)
WARNING *** file size (195657) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 27

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 27행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2604_영도구_부산광역시.xls (지역: 영도구)
WARNING *** file size (170804) no

In [15]:
busan_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n오거돈,한나라당\n허남식,민주노동당\n김석준,column_8,계,무효\n투표수,기권수,column_12,column_13,column_14
0,중구,NaN,합계,"43,062",NaN,"22,768","5,155","15,325","2,047",NaN,"22,527",241,"20,294",NaN,NaN,0.0
1,서구,NaN,합계,"112,683",NaN,"55,046","11,782","37,607","5,038",NaN,"54,427",619,"57,637",NaN,NaN,0.0
2,동구,NaN,합계,"89,864",NaN,"46,773","10,749","30,690","4,592",NaN,"46,031",742,"43,091",NaN,NaN,0.0
3,영도구,NaN,합계,"131,447",NaN,"63,268","17,638","38,017","6,857",NaN,"62,512",756,"68,179",NaN,NaN,0.0
4,부산진구,NaN,합계,"325,963",NaN,"157,499","35,746","103,671","16,643",NaN,"156,060","1,439","168,464",NaN,NaN,0.0
5,동래구,NaN,합계,"216,997",NaN,"107,497","23,133","73,580","10,026",NaN,"106,739",758,"109,500",NaN,NaN,0.0
6,남구,NaN,합계,"238,368",NaN,"116,840","28,203","75,331","12,390",NaN,"115,924",916,"121,528",NaN,NaN,0.0
7,북구,NaN,합계,"248,396",NaN,"117,428","29,620","73,619","13,223",NaN,"116,462",966,"130,968",NaN,NaN,0.0
8,해운대구,NaN,합계,"309,668",NaN,"145,642","35,920","93,928","14,637",NaN,"144,485","1,157","164,026",NaN,NaN,0.0
9,기장군,NaN,합계,"61,953",NaN,"36,135","8,326","23,264","3,995",NaN,"35,585",550,"25,818",NaN,NaN,0.0


In [16]:
busan_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n오거돈',
 '한나라당\n허남식',
 '민주노동당\n김석준',
 'column_8',
 '계',
 '무효\n투표수',
 '기권수',
 'column_12',
 'column_13',
 'column_14']

In [17]:
rename_busan = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n오거돈': '득표수_1_열린우리당_오거돈',
    '한나라당\n허남식': '득표수_2_한나라당_허남식',
    '민주노동당\n김석준': '득표수_4_민주노동당_김석준',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [18]:
busan_4th = busan_4th.rename(columns=rename_busan).drop(columns=['column_0', '읍면동명', 'column_3', 'column_8', 'column_12', 'column_13', 'column_14'])
busan_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_오거돈,득표수_2_한나라당_허남식,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수
0,중구,"43,062","22,768","5,155","15,325","2,047","22,527",241,"20,294"
1,서구,"112,683","55,046","11,782","37,607","5,038","54,427",619,"57,637"
2,동구,"89,864","46,773","10,749","30,690","4,592","46,031",742,"43,091"
3,영도구,"131,447","63,268","17,638","38,017","6,857","62,512",756,"68,179"
4,부산진구,"325,963","157,499","35,746","103,671","16,643","156,060","1,439","168,464"
5,동래구,"216,997","107,497","23,133","73,580","10,026","106,739",758,"109,500"
6,남구,"238,368","116,840","28,203","75,331","12,390","115,924",916,"121,528"
7,북구,"248,396","117,428","29,620","73,619","13,223","116,462",966,"130,968"
8,해운대구,"309,668","145,642","35,920","93,928","14,637","144,485","1,157","164,026"
9,기장군,"61,953","36,135","8,326","23,264","3,995","35,585",550,"25,818"


In [19]:
busan_4th = busan_4th.assign(
    시도='부산광역시'
)[['시도'] + busan_4th.columns.tolist()]

In [20]:
busan_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오거돈,득표수_2_한나라당_허남식,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,중구,"43,062","22,768","5,155","15,325","2,047","22,527",241,"20,294"
1,부산광역시,서구,"112,683","55,046","11,782","37,607","5,038","54,427",619,"57,637"
2,부산광역시,동구,"89,864","46,773","10,749","30,690","4,592","46,031",742,"43,091"
3,부산광역시,영도구,"131,447","63,268","17,638","38,017","6,857","62,512",756,"68,179"
4,부산광역시,부산진구,"325,963","157,499","35,746","103,671","16,643","156,060","1,439","168,464"


In [21]:
busan_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               16 non-null     object
 1   구시군              16 non-null     object
 2   선거인수             16 non-null     object
 3   투표수              16 non-null     object
 4   득표수_1_열린우리당_오거돈  16 non-null     object
 5   득표수_2_한나라당_허남식   16 non-null     object
 6   득표수_4_민주노동당_김석준  16 non-null     object
 7   득표수_계            16 non-null     object
 8   무효투표수            16 non-null     object
 9   기권수              16 non-null     object
dtypes: object(10)
memory usage: 1.4+ KB


In [22]:
busan_4th = busan_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [23]:
# 수치형 열만 합계 구하기
summary_row = busan_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '부산광역시')

# summary_row를 맨 위에 붙이기
busan_4th_with_total = pd.concat([summary_row, busan_4th], ignore_index=True)

In [24]:
busan_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오거돈,득표수_2_한나라당_허남식,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,합계,2845104,1378618,329470,895214,141061,1365745,12873,1466486
1,부산광역시,중구,43062,22768,5155,15325,2047,22527,241,20294
2,부산광역시,서구,112683,55046,11782,37607,5038,54427,619,57637
3,부산광역시,동구,89864,46773,10749,30690,4592,46031,742,43091
4,부산광역시,영도구,131447,63268,17638,38017,6857,62512,756,68179
5,부산광역시,부산진구,325963,157499,35746,103671,16643,156060,1439,168464
6,부산광역시,동래구,216997,107497,23133,73580,10026,106739,758,109500
7,부산광역시,남구,238368,116840,28203,75331,12390,115924,916,121528
8,부산광역시,북구,248396,117428,29620,73619,13223,116462,966,130968
9,부산광역시,해운대구,309668,145642,35920,93928,14637,144485,1157,164026


In [25]:
busan_4th_with_total.to_csv("temp1_governor_busan_4.csv", index=False, encoding="utf-8-sig")


## Daegu


In [26]:
daegu_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/27_대구"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/27_대구
------------------------------------------------------------
폴더에서 8개 파일 발견

처리 중: 2701_중구_대구광역시.xls (지역: 중구)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 17
데이터 행 수: 23

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 23행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2702_동구_대구광역시.xls (지역: 동구)
WARNING *** file size (139687) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 17
데이터 행 수: 30

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 30행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2703_서구_대구광역시.xls (지역: 서구)
WARNING *** file size (118590) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 17
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2704_남구_대구광역시.xls (지역: 남구)
WARNING *** file size (103070) not 5

In [27]:
daegu_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n이재용,한나라당\n김범일,민주노동당\n이연재,국민중심당\n박승국,무소속\n백승홍,column_10,계,무효\n투표수,기권수,column_14,column_15,column_16
0,중구,NaN,합계,"66,126",NaN,"35,274","7,753","24,056",960,228,"1,870",NaN,"34,867",407,"30,852",NaN,NaN,0.0
1,동구,NaN,합계,"267,710",NaN,"129,221","27,773","90,128","4,935","1,291","3,714",NaN,"127,841","1,380","138,489",NaN,NaN,0.0
2,서구,NaN,합계,"194,812",NaN,"92,272","16,137","63,241","3,304",807,"7,757",NaN,"91,246","1,026","102,540",NaN,NaN,0.0
3,남구,NaN,합계,"145,954",NaN,"70,026","17,919","47,545","1,876",387,"1,706",NaN,"69,433",593,"75,928",NaN,NaN,0.0
4,북구,NaN,합계,"341,531",NaN,"159,678","32,906","110,959","6,687","2,731","5,025",NaN,"158,308","1,370","181,853",NaN,NaN,0.0
5,수성구,NaN,합계,"320,246",NaN,"157,451","31,146","114,184","5,563","1,090","4,375",NaN,"156,358","1,093","162,795",NaN,NaN,0.0
6,달서구,NaN,합계,"432,607",NaN,"206,965","43,630","145,236","7,956","1,331","7,388",NaN,"205,541","1,424","225,642",NaN,NaN,0.0
7,달성군,NaN,합계,"116,057",NaN,"64,173","13,867","40,708","4,216",899,"3,397",NaN,"63,087","1,086","51,884",NaN,NaN,0.0


In [28]:
daegu_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n이재용',
 '한나라당\n김범일',
 '민주노동당\n이연재',
 '국민중심당\n박승국',
 '무소속\n백승홍',
 'column_10',
 '계',
 '무효\n투표수',
 '기권수',
 'column_14',
 'column_15',
 'column_16']

In [29]:
rename_daegu = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n이재용': '득표수_1_열린우리당_이재용',
    '한나라당\n김범일': '득표수_2_한나라당_김범일',
    '민주노동당\n이연재': '득표수_4_민주노동당_이연재',
    '국민중심당\n박승국': '득표수_5_국민중심당_박승국',
    '무소속\n백승홍': '득표수_6_무소속_백승홍',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [30]:
daegu_4th = daegu_4th.rename(columns=rename_daegu).drop(columns=['column_0', '읍면동명', 'column_3', 'column_10', 'column_14', 'column_15', 'column_16'])
daegu_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_이재용,득표수_2_한나라당_김범일,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수
0,중구,"66,126","35,274","7,753","24,056",960,228,"1,870","34,867",407,"30,852"
1,동구,"267,710","129,221","27,773","90,128","4,935","1,291","3,714","127,841","1,380","138,489"
2,서구,"194,812","92,272","16,137","63,241","3,304",807,"7,757","91,246","1,026","102,540"
3,남구,"145,954","70,026","17,919","47,545","1,876",387,"1,706","69,433",593,"75,928"
4,북구,"341,531","159,678","32,906","110,959","6,687","2,731","5,025","158,308","1,370","181,853"
5,수성구,"320,246","157,451","31,146","114,184","5,563","1,090","4,375","156,358","1,093","162,795"
6,달서구,"432,607","206,965","43,630","145,236","7,956","1,331","7,388","205,541","1,424","225,642"
7,달성군,"116,057","64,173","13,867","40,708","4,216",899,"3,397","63,087","1,086","51,884"


In [31]:
daegu_4th = daegu_4th.assign(
    시도='대구광역시'
)[['시도'] + daegu_4th.columns.tolist()]

In [32]:
daegu_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이재용,득표수_2_한나라당_김범일,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수
0,대구광역시,중구,"66,126","35,274","7,753","24,056",960,228,"1,870","34,867",407,"30,852"
1,대구광역시,동구,"267,710","129,221","27,773","90,128","4,935","1,291","3,714","127,841","1,380","138,489"
2,대구광역시,서구,"194,812","92,272","16,137","63,241","3,304",807,"7,757","91,246","1,026","102,540"
3,대구광역시,남구,"145,954","70,026","17,919","47,545","1,876",387,"1,706","69,433",593,"75,928"
4,대구광역시,북구,"341,531","159,678","32,906","110,959","6,687","2,731","5,025","158,308","1,370","181,853"


In [33]:
daegu_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               8 non-null      object
 1   구시군              8 non-null      object
 2   선거인수             8 non-null      object
 3   투표수              8 non-null      object
 4   득표수_1_열린우리당_이재용  8 non-null      object
 5   득표수_2_한나라당_김범일   8 non-null      object
 6   득표수_4_민주노동당_이연재  8 non-null      object
 7   득표수_5_국민중심당_박승국  8 non-null      object
 8   득표수_6_무소속_백승홍    8 non-null      object
 9   득표수_계            8 non-null      object
 10  무효투표수            8 non-null      object
 11  기권수              8 non-null      object
dtypes: object(12)
memory usage: 900.0+ bytes


In [34]:
daegu_4th = daegu_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [35]:
# 수치형 열만 합계 구하기
summary_row = daegu_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대구광역시')

# summary_row를 맨 위에 붙이기
daegu_4th_with_total = pd.concat([summary_row, daegu_4th], ignore_index=True)

In [36]:
daegu_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이재용,득표수_2_한나라당_김범일,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수
0,대구광역시,합계,1885043,915060,191131,636057,35497,8764,35232,906681,8379,969983
1,대구광역시,중구,66126,35274,7753,24056,960,228,1870,34867,407,30852
2,대구광역시,동구,267710,129221,27773,90128,4935,1291,3714,127841,1380,138489
3,대구광역시,서구,194812,92272,16137,63241,3304,807,7757,91246,1026,102540
4,대구광역시,남구,145954,70026,17919,47545,1876,387,1706,69433,593,75928
5,대구광역시,북구,341531,159678,32906,110959,6687,2731,5025,158308,1370,181853
6,대구광역시,수성구,320246,157451,31146,114184,5563,1090,4375,156358,1093,162795
7,대구광역시,달서구,432607,206965,43630,145236,7956,1331,7388,205541,1424,225642
8,대구광역시,달성군,116057,64173,13867,40708,4216,899,3397,63087,1086,51884


In [37]:
daegu_4th_with_total.to_csv("temp1_governor_daegu_4.csv", index=False, encoding="utf-8-sig")


## Incheon


In [38]:
incheon_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/28_인천"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/28_인천
------------------------------------------------------------
폴더에서 10개 파일 발견

처리 중: 2801_중구_인천광역시.xls (지역: 중구)
WARNING *** file size (96823) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 12

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 12행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2802_동구_인천광역시.xls (지역: 동구)
WARNING *** file size (119002) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 21

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 21행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2803_남구_인천광역시.xls (지역: 남구)
WARNING *** file size (207669) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 34

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 34행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

In [39]:
incheon_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n최기선,한나라당\n안상수,민주당\n신경철,민주노동당\n김성진,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,중구,NaN,합계,"72,876",NaN,"34,796","8,119","21,570","1,877","2,764",NaN,"34,330",466,"38,080",NaN,NaN,0.0
1,동구,NaN,합계,"61,515",NaN,"30,794","6,663","18,707","1,483","3,514",NaN,"30,367",427,"30,721",NaN,NaN,0.0
2,남구,NaN,합계,"327,614",NaN,"140,581","31,036","90,299","6,402","11,403",NaN,"139,140","1,441","187,033",NaN,NaN,0.0
3,연수구,NaN,합계,"191,076",NaN,"88,010","19,370","57,235","2,892","7,873",NaN,"87,370",640,"103,066",NaN,NaN,0.0
4,남동구,NaN,합계,"284,121",NaN,"123,423","28,961","73,284","7,794","12,285",NaN,"122,324","1,099","160,698",NaN,NaN,0.0
5,부평구,NaN,합계,"425,004",NaN,"181,447","42,156","106,584","11,245","19,942",NaN,"179,927","1,520","243,557",NaN,NaN,0.0
6,계양구,NaN,합계,"239,585",NaN,"100,411","26,480","58,806","5,363","8,963",NaN,"99,612",799,"139,174",NaN,NaN,0.0
7,서구,NaN,합계,"271,645",NaN,"114,473","27,258","70,169","6,074","9,834",NaN,"113,335","1,138","157,172",NaN,NaN,0.0
8,강화군,NaN,합계,"53,699",NaN,"35,405","8,071","23,919",860,"1,700",NaN,"34,550",855,"18,294",NaN,NaN,0.0
9,옹진군,NaN,합계,"13,268",NaN,"10,166","2,536","6,359",349,620,NaN,"9,864",302,"3,102",NaN,NaN,0.0


In [40]:
incheon_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n최기선',
 '한나라당\n안상수',
 '민주당\n신경철',
 '민주노동당\n김성진',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [41]:
rename_incheon = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n최기선': '득표수_1_열린우리당_최기선',
    '한나라당\n안상수': '득표수_2_한나라당_안상수',
    '민주당\n신경철': '득표수_3_민주당_신경철',
    '민주노동당\n김성진': '득표수_4_민주노동당_김성진',
    '무소속\n백승홍': '득표수_6_무소속_백승홍',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [42]:
incheon_4th = incheon_4th.rename(columns=rename_incheon).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
incheon_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_최기선,득표수_2_한나라당_안상수,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수
0,중구,"72,876","34,796","8,119","21,570","1,877","2,764","34,330",466,"38,080"
1,동구,"61,515","30,794","6,663","18,707","1,483","3,514","30,367",427,"30,721"
2,남구,"327,614","140,581","31,036","90,299","6,402","11,403","139,140","1,441","187,033"
3,연수구,"191,076","88,010","19,370","57,235","2,892","7,873","87,370",640,"103,066"
4,남동구,"284,121","123,423","28,961","73,284","7,794","12,285","122,324","1,099","160,698"
5,부평구,"425,004","181,447","42,156","106,584","11,245","19,942","179,927","1,520","243,557"
6,계양구,"239,585","100,411","26,480","58,806","5,363","8,963","99,612",799,"139,174"
7,서구,"271,645","114,473","27,258","70,169","6,074","9,834","113,335","1,138","157,172"
8,강화군,"53,699","35,405","8,071","23,919",860,"1,700","34,550",855,"18,294"
9,옹진군,"13,268","10,166","2,536","6,359",349,620,"9,864",302,"3,102"


In [43]:
incheon_4th = incheon_4th.assign(
    시도='인천광역시'
)[['시도'] + incheon_4th.columns.tolist()]

In [44]:
incheon_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_최기선,득표수_2_한나라당_안상수,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수
0,인천광역시,중구,"72,876","34,796","8,119","21,570","1,877","2,764","34,330",466,"38,080"
1,인천광역시,동구,"61,515","30,794","6,663","18,707","1,483","3,514","30,367",427,"30,721"
2,인천광역시,남구,"327,614","140,581","31,036","90,299","6,402","11,403","139,140","1,441","187,033"
3,인천광역시,연수구,"191,076","88,010","19,370","57,235","2,892","7,873","87,370",640,"103,066"
4,인천광역시,남동구,"284,121","123,423","28,961","73,284","7,794","12,285","122,324","1,099","160,698"


In [45]:
incheon_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               10 non-null     object
 1   구시군              10 non-null     object
 2   선거인수             10 non-null     object
 3   투표수              10 non-null     object
 4   득표수_1_열린우리당_최기선  10 non-null     object
 5   득표수_2_한나라당_안상수   10 non-null     object
 6   득표수_3_민주당_신경철    10 non-null     object
 7   득표수_4_민주노동당_김성진  10 non-null     object
 8   득표수_계            10 non-null     object
 9   무효투표수            10 non-null     object
 10  기권수              10 non-null     object
dtypes: object(11)
memory usage: 1012.0+ bytes


In [46]:
incheon_4th = incheon_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [47]:
# 수치형 열만 합계 구하기
summary_row = incheon_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '인천광역시')

# summary_row를 맨 위에 붙이기
incheon_4th_with_total = pd.concat([summary_row, incheon_4th], ignore_index=True)

In [48]:
incheon_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_최기선,득표수_2_한나라당_안상수,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수
0,인천광역시,합계,1940403,859506,200650,526932,44339,78898,850819,8687,1080897
1,인천광역시,중구,72876,34796,8119,21570,1877,2764,34330,466,38080
2,인천광역시,동구,61515,30794,6663,18707,1483,3514,30367,427,30721
3,인천광역시,남구,327614,140581,31036,90299,6402,11403,139140,1441,187033
4,인천광역시,연수구,191076,88010,19370,57235,2892,7873,87370,640,103066
5,인천광역시,남동구,284121,123423,28961,73284,7794,12285,122324,1099,160698
6,인천광역시,부평구,425004,181447,42156,106584,11245,19942,179927,1520,243557
7,인천광역시,계양구,239585,100411,26480,58806,5363,8963,99612,799,139174
8,인천광역시,서구,271645,114473,27258,70169,6074,9834,113335,1138,157172
9,인천광역시,강화군,53699,35405,8071,23919,860,1700,34550,855,18294


In [49]:
incheon_4th_with_total.to_csv("temp1_governor_incheon_4.csv", index=False, encoding="utf-8-sig")


## Gwangju


In [50]:
gwangju_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/29_광주"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/29_광주
------------------------------------------------------------
폴더에서 5개 파일 발견

처리 중: 2901_동구_광주광역시.xls (지역: 동구)
WARNING *** file size (132977) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 23

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 23행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2902_서구_광주광역시.xls (지역: 서구)
WARNING *** file size (160005) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 27

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 27행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 2903_남구_광주광역시.xls (지역: 남구)
WARNING *** file size (153225) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

In [51]:
gwangju_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n조영택,한나라당\n한영,민주당\n박광태,민주노동당\n오병윤,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,동구,NaN,합계,"90,767",NaN,"43,236","16,085","2,201","21,204","3,240",NaN,"42,730",506,"47,531",NaN,NaN,0.0
1,서구,NaN,합계,"221,034",NaN,"102,566","34,849","3,911","51,556","11,386",NaN,"101,702",864,"118,468",NaN,NaN,0.0
2,남구,NaN,합계,"161,406",NaN,"78,309","26,536","3,011","42,078","5,861",NaN,"77,486",823,"83,097",NaN,NaN,0.0
3,북구,NaN,합계,"333,413",NaN,"147,702","53,265","5,905","72,580","14,636",NaN,"146,386","1,316","185,711",NaN,NaN,0.0
4,광산구,NaN,합계,"208,000",NaN,"97,503","27,021","3,433","52,466","13,494",NaN,"96,414","1,089","110,497",NaN,NaN,0.0


In [52]:
gwangju_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n조영택',
 '한나라당\n한영',
 '민주당\n박광태',
 '민주노동당\n오병윤',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [53]:
rename_gwangju = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n조영택': '득표수_1_열린우리당_조영택',
    '한나라당\n한영': '득표수_2_한나라당_한영',
    '민주당\n박광태': '득표수_3_민주당_박광태',
    '민주노동당\n오병윤': '득표수_4_민주노동당_오병윤',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [54]:
gwangju_4th = gwangju_4th.rename(columns=rename_gwangju).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
gwangju_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_조영택,득표수_2_한나라당_한영,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수
0,동구,"90,767","43,236","16,085","2,201","21,204","3,240","42,730",506,"47,531"
1,서구,"221,034","102,566","34,849","3,911","51,556","11,386","101,702",864,"118,468"
2,남구,"161,406","78,309","26,536","3,011","42,078","5,861","77,486",823,"83,097"
3,북구,"333,413","147,702","53,265","5,905","72,580","14,636","146,386","1,316","185,711"
4,광산구,"208,000","97,503","27,021","3,433","52,466","13,494","96,414","1,089","110,497"


In [55]:
gwangju_4th = gwangju_4th.assign(
    시도='광주광역시'
)[['시도'] + gwangju_4th.columns.tolist()]

In [56]:
gwangju_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_조영택,득표수_2_한나라당_한영,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수
0,광주광역시,동구,"90,767","43,236","16,085","2,201","21,204","3,240","42,730",506,"47,531"
1,광주광역시,서구,"221,034","102,566","34,849","3,911","51,556","11,386","101,702",864,"118,468"
2,광주광역시,남구,"161,406","78,309","26,536","3,011","42,078","5,861","77,486",823,"83,097"
3,광주광역시,북구,"333,413","147,702","53,265","5,905","72,580","14,636","146,386","1,316","185,711"
4,광주광역시,광산구,"208,000","97,503","27,021","3,433","52,466","13,494","96,414","1,089","110,497"


In [57]:
gwangju_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               5 non-null      object
 1   구시군              5 non-null      object
 2   선거인수             5 non-null      object
 3   투표수              5 non-null      object
 4   득표수_1_열린우리당_조영택  5 non-null      object
 5   득표수_2_한나라당_한영    5 non-null      object
 6   득표수_3_민주당_박광태    5 non-null      object
 7   득표수_4_민주노동당_오병윤  5 non-null      object
 8   득표수_계            5 non-null      object
 9   무효투표수            5 non-null      object
 10  기권수              5 non-null      object
dtypes: object(11)
memory usage: 572.0+ bytes


In [58]:
gwangju_4th = gwangju_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [59]:
# 수치형 열만 합계 구하기
summary_row = gwangju_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '광주광역시')

# summary_row를 맨 위에 붙이기
gwangju_4th_with_total = pd.concat([summary_row, gwangju_4th], ignore_index=True)

In [60]:
gwangju_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_조영택,득표수_2_한나라당_한영,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수
0,광주광역시,합계,1014620,469316,157756,18461,239884,48617,464718,4598,545304
1,광주광역시,동구,90767,43236,16085,2201,21204,3240,42730,506,47531
2,광주광역시,서구,221034,102566,34849,3911,51556,11386,101702,864,118468
3,광주광역시,남구,161406,78309,26536,3011,42078,5861,77486,823,83097
4,광주광역시,북구,333413,147702,53265,5905,72580,14636,146386,1316,185711
5,광주광역시,광산구,208000,97503,27021,3433,52466,13494,96414,1089,110497


In [61]:
gwangju_4th_with_total.to_csv("temp1_governor_gwangju_4.csv", index=False, encoding="utf-8-sig")


## Daejeon


In [62]:
daejeon_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/30_대전"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/30_대전
------------------------------------------------------------
폴더에서 5개 파일 발견

처리 중: 3001_동구_대전광역시.xls (지역: 동구)
WARNING *** file size (102063) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 18
데이터 행 수: 31

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 31행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 3002_중구_대전광역시.xls (지역: 중구)
WARNING *** file size (84434) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 18
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 3003_서구_대전광역시.xls (지역: 서구)
WARNING *** file size (105913) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 18
데이터 행 수: 32

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 32행 → 필터링 후: 1행
  ✓ 완료: 1행 추가


In [63]:
daejeon_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n염홍철,한나라당\n박성효,민주당\n최기복,민주노동당\n박춘호,국민중심당\n남충희,한미준\n고낙정,column_11,계,무효\n투표수,기권수,column_15,column_16,column_17
0,동구,NaN,합계,"183,206",NaN,"87,967","36,078","35,745","1,262","2,401","10,736",685,NaN,"86,907","1,060","95,239",NaN,NaN,0.0
1,중구,NaN,합계,"203,076",NaN,"100,902","39,961","44,780","1,202","2,324","11,290",456,NaN,"100,013",889,"102,174",NaN,NaN,0.0
2,서구,NaN,합계,"367,265",NaN,"182,493","73,194","83,617","2,040","4,025","17,622",759,NaN,"181,257","1,236","184,772",NaN,NaN,0.0
3,유성구,NaN,합계,"164,867",NaN,"84,058","36,081","36,053",873,"2,979","7,204",354,NaN,"83,544",514,"80,809",NaN,NaN,0.0
4,대덕구,NaN,합계,"159,054",NaN,"77,148","31,959","31,294","1,065","3,170","8,379",453,NaN,"76,320",828,"81,906",NaN,NaN,0.0


In [64]:
daejeon_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n염홍철',
 '한나라당\n박성효',
 '민주당\n최기복',
 '민주노동당\n박춘호',
 '국민중심당\n남충희',
 '한미준\n고낙정',
 'column_11',
 '계',
 '무효\n투표수',
 '기권수',
 'column_15',
 'column_16',
 'column_17']

In [65]:
rename_daejeon = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n염홍철': '득표수_1_열린우리당_염홍철',
    '한나라당\n박성효': '득표수_2_한나라당_박성효',
    '민주노동당\n박춘호': '득표수_4_민주노동당_박춘호',
    '국민중심당\n남충희': '득표수_5_국민중심당_남충희',
    '한미준\n고낙정': '득표수_7_한국의미래를준비하는당_고낙정',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [66]:
daejeon_4th = daejeon_4th.rename(columns=rename_daejeon).drop(columns=['column_0', '읍면동명', 'column_3', 'column_11', 'column_15', 'column_16', 'column_17'])
daejeon_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_염홍철,득표수_2_한나라당_박성효,민주당\n최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수
0,동구,"183,206","87,967","36,078","35,745","1,262","2,401","10,736",685,"86,907","1,060","95,239"
1,중구,"203,076","100,902","39,961","44,780","1,202","2,324","11,290",456,"100,013",889,"102,174"
2,서구,"367,265","182,493","73,194","83,617","2,040","4,025","17,622",759,"181,257","1,236","184,772"
3,유성구,"164,867","84,058","36,081","36,053",873,"2,979","7,204",354,"83,544",514,"80,809"
4,대덕구,"159,054","77,148","31,959","31,294","1,065","3,170","8,379",453,"76,320",828,"81,906"


In [67]:
daejeon_4th = daejeon_4th.assign(
    시도='대전광역시'
)[['시도'] + daejeon_4th.columns.tolist()]

In [68]:
daejeon_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_염홍철,득표수_2_한나라당_박성효,민주당\n최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수
0,대전광역시,동구,"183,206","87,967","36,078","35,745","1,262","2,401","10,736",685,"86,907","1,060","95,239"
1,대전광역시,중구,"203,076","100,902","39,961","44,780","1,202","2,324","11,290",456,"100,013",889,"102,174"
2,대전광역시,서구,"367,265","182,493","73,194","83,617","2,040","4,025","17,622",759,"181,257","1,236","184,772"
3,대전광역시,유성구,"164,867","84,058","36,081","36,053",873,"2,979","7,204",354,"83,544",514,"80,809"
4,대전광역시,대덕구,"159,054","77,148","31,959","31,294","1,065","3,170","8,379",453,"76,320",828,"81,906"


In [69]:
daejeon_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   시도                     5 non-null      object
 1   구시군                    5 non-null      object
 2   선거인수                   5 non-null      object
 3   투표수                    5 non-null      object
 4   득표수_1_열린우리당_염홍철        5 non-null      object
 5   득표수_2_한나라당_박성효         5 non-null      object
 6   민주당
최기복                5 non-null      object
 7   득표수_4_민주노동당_박춘호        5 non-null      object
 8   득표수_5_국민중심당_남충희        5 non-null      object
 9   득표수_7_한국의미래를준비하는당_고낙정  5 non-null      object
 10  득표수_계                  5 non-null      object
 11  무효투표수                  5 non-null      object
 12  기권수                    5 non-null      object
dtypes: object(13)
memory usage: 652.0+ bytes


In [70]:
daejeon_4th = daejeon_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [71]:
# 수치형 열만 합계 구하기
summary_row = daejeon_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대전광역시')

# summary_row를 맨 위에 붙이기
daejeon_4th_with_total = pd.concat([summary_row, daejeon_4th], ignore_index=True)

In [72]:
daejeon_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_염홍철,득표수_2_한나라당_박성효,민주당\n최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수
0,대전광역시,합계,1077468,532568,217273,231489,6442,14899,55231,2707,528041,4527,544900
1,대전광역시,동구,183206,87967,36078,35745,1262,2401,10736,685,86907,1060,95239
2,대전광역시,중구,203076,100902,39961,44780,1202,2324,11290,456,100013,889,102174
3,대전광역시,서구,367265,182493,73194,83617,2040,4025,17622,759,181257,1236,184772
4,대전광역시,유성구,164867,84058,36081,36053,873,2979,7204,354,83544,514,80809
5,대전광역시,대덕구,159054,77148,31959,31294,1065,3170,8379,453,76320,828,81906


In [73]:
daejeon_4th_with_total.to_csv("temp1_governor_daejeon_4.csv", index=False, encoding="utf-8-sig")


## Ulsan


In [74]:
ulsan_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/31_울산"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/31_울산
------------------------------------------------------------
폴더에서 5개 파일 발견

처리 중: 3101_중구_울산광역시.xls (지역: 중구)
WARNING *** file size (170774) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 24

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 24행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 3102_남구_울산광역시.xls (지역: 남구)
WARNING *** file size (170826) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 24

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 24행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 3103_동구_울산광역시.xls (지역: 동구)
WARNING *** file size (126389) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 13

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 13행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

In [75]:
ulsan_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n심규명,한나라당\n박맹우,민주노동당\n노옥희,column_8,계,무효\n투표수,기권수,column_12,column_13,column_14
0,중구,NaN,합계,"175,337",NaN,"88,549","9,023","59,499","19,390",NaN,"87,912",637,"86,788",NaN,NaN,0.0
1,남구,NaN,합계,"250,864",NaN,"123,837","14,533","85,037","23,446",NaN,"123,016",821,"127,027",NaN,NaN,0.0
2,동구,NaN,합계,"136,957",NaN,"75,733","8,675","40,077","26,290",NaN,"75,042",691,"61,224",NaN,NaN,0.0
3,북구,NaN,합계,"99,489",NaN,"55,385","5,176","28,677","21,047",NaN,"54,900",485,"44,104",NaN,NaN,0.0
4,울주군,NaN,합계,"127,642",NaN,"73,696","10,172","48,071","14,211",NaN,"72,454","1,242","53,946",NaN,NaN,0.0


In [76]:
ulsan_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n심규명',
 '한나라당\n박맹우',
 '민주노동당\n노옥희',
 'column_8',
 '계',
 '무효\n투표수',
 '기권수',
 'column_12',
 'column_13',
 'column_14']

In [77]:
rename_ulsan = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n심규명': '득표수_1_열린우리당_심규명',
    '한나라당\n박맹우': '득표수_2_한나라당_박맹우',
    '민주노동당\n노옥희': '득표수_4_민주노동당_노옥희',
    '무소속\n백승홍': '득표수_6_무소속_백승홍',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [78]:
ulsan_4th = ulsan_4th.rename(columns=rename_ulsan).drop(columns=['column_0', '읍면동명', 'column_3', 'column_8', 'column_12', 'column_13', 'column_14'])
ulsan_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_심규명,득표수_2_한나라당_박맹우,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수
0,중구,"175,337","88,549","9,023","59,499","19,390","87,912",637,"86,788"
1,남구,"250,864","123,837","14,533","85,037","23,446","123,016",821,"127,027"
2,동구,"136,957","75,733","8,675","40,077","26,290","75,042",691,"61,224"
3,북구,"99,489","55,385","5,176","28,677","21,047","54,900",485,"44,104"
4,울주군,"127,642","73,696","10,172","48,071","14,211","72,454","1,242","53,946"


In [79]:
ulsan_4th = ulsan_4th.assign(
    시도='울산광역시'
)[['시도'] + ulsan_4th.columns.tolist()]

In [80]:
ulsan_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_심규명,득표수_2_한나라당_박맹우,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수
0,울산광역시,중구,"175,337","88,549","9,023","59,499","19,390","87,912",637,"86,788"
1,울산광역시,남구,"250,864","123,837","14,533","85,037","23,446","123,016",821,"127,027"
2,울산광역시,동구,"136,957","75,733","8,675","40,077","26,290","75,042",691,"61,224"
3,울산광역시,북구,"99,489","55,385","5,176","28,677","21,047","54,900",485,"44,104"
4,울산광역시,울주군,"127,642","73,696","10,172","48,071","14,211","72,454","1,242","53,946"


In [81]:
ulsan_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               5 non-null      object
 1   구시군              5 non-null      object
 2   선거인수             5 non-null      object
 3   투표수              5 non-null      object
 4   득표수_1_열린우리당_심규명  5 non-null      object
 5   득표수_2_한나라당_박맹우   5 non-null      object
 6   득표수_4_민주노동당_노옥희  5 non-null      object
 7   득표수_계            5 non-null      object
 8   무효투표수            5 non-null      object
 9   기권수              5 non-null      object
dtypes: object(10)
memory usage: 532.0+ bytes


In [82]:
ulsan_4th = ulsan_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [83]:
# 수치형 열만 합계 구하기
summary_row = ulsan_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '울산광역시')

# summary_row를 맨 위에 붙이기
ulsan_4th_with_total = pd.concat([summary_row, ulsan_4th], ignore_index=True)

In [84]:
ulsan_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_심규명,득표수_2_한나라당_박맹우,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수
0,울산광역시,합계,790289,417200,47579,261361,104384,413324,3876,373089
1,울산광역시,중구,175337,88549,9023,59499,19390,87912,637,86788
2,울산광역시,남구,250864,123837,14533,85037,23446,123016,821,127027
3,울산광역시,동구,136957,75733,8675,40077,26290,75042,691,61224
4,울산광역시,북구,99489,55385,5176,28677,21047,54900,485,44104
5,울산광역시,울주군,127642,73696,10172,48071,14211,72454,1242,53946


In [85]:
ulsan_4th_with_total.to_csv("temp1_governor_ulsan_4.csv", index=False, encoding="utf-8-sig")


## Gyeonggi


In [86]:
gyeonggi_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/41_경기"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/41_경기
------------------------------------------------------------
폴더에서 44개 파일 발견

처리 중: 4101_수원시장안구_경기도.xls (지역: 수원시장안구)
WARNING *** file size (103618) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 13

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 13행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4102_수원시권선구_경기도.xls (지역: 수원시권선구)
WARNING *** file size (103701) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 13

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 13행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4103_수원시팔달구_경기도.xls (지역: 수원시팔달구)
WARNING *** file size (125786) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 22

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 22행 → 필터링 

In [87]:
gyeonggi_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n진대제,한나라당\n김문수,민주당\n박정일,민주노동당\n김용한,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,수원시장안구,NaN,합계,"206,015",NaN,"98,094","31,861","56,361","3,366","5,795",NaN,"97,383",711,"107,921",NaN,NaN,0.0
1,수원시권선구,NaN,합계,"217,455",NaN,"97,485","31,707","56,267","2,822","5,838",NaN,"96,634",851,"119,970",NaN,NaN,0.0
2,수원시팔달구,NaN,합계,"166,662",NaN,"72,030","22,480","42,829","2,501","3,520",NaN,"71,330",700,"94,632",NaN,NaN,0.0
3,수원시영통구,NaN,합계,"173,454",NaN,"82,006","31,247","45,246","1,653","3,442",NaN,"81,588",418,"91,448",NaN,NaN,0.0
4,성남시수정구,NaN,합계,"206,678",NaN,"84,853","28,638","41,683","7,413","6,071",NaN,"83,805","1,048","121,825",NaN,NaN,0.0
5,성남시중원구,NaN,합계,"207,205",NaN,"82,650","26,147","41,203","7,825","6,653",NaN,"81,828",822,"124,555",NaN,NaN,0.0
6,성남시분당구,NaN,합계,"330,404",NaN,"164,190","46,797","107,353","4,380","4,789",NaN,"163,319",871,"166,214",NaN,NaN,0.0
7,의정부시,NaN,합계,"299,556",NaN,"130,381","41,367","77,107","3,905","6,738",NaN,"129,117","1,264","169,175",NaN,NaN,0.0
8,안양시만안구,NaN,합계,"204,688",NaN,"95,595","31,436","53,699","4,500","5,055",NaN,"94,690",905,"109,093",NaN,NaN,0.0
9,안양시동안구,NaN,합계,"259,321",NaN,"134,635","46,671","76,522","4,696","5,789",NaN,"133,678",957,"124,686",NaN,NaN,0.0


In [88]:
gyeonggi_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n진대제',
 '한나라당\n김문수',
 '민주당\n박정일',
 '민주노동당\n김용한',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [89]:
rename_gyeonggi = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n진대제': '득표수_1_열린우리당_진대제',
    '한나라당\n김문수': '득표수_2_한나라당_김문수',
    '민주당\n박정일': '득표수_3_민주당_최기복',
    '민주노동당\n김용한': '득표수_4_민주노동당_김용한',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [90]:
gyeonggi_4th = gyeonggi_4th.rename(columns=rename_gyeonggi).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
gyeonggi_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_진대제,득표수_2_한나라당_김문수,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수
0,수원시장안구,"206,015","98,094","31,861","56,361","3,366","5,795","97,383",711,"107,921"
1,수원시권선구,"217,455","97,485","31,707","56,267","2,822","5,838","96,634",851,"119,970"
2,수원시팔달구,"166,662","72,030","22,480","42,829","2,501","3,520","71,330",700,"94,632"
3,수원시영통구,"173,454","82,006","31,247","45,246","1,653","3,442","81,588",418,"91,448"
4,성남시수정구,"206,678","84,853","28,638","41,683","7,413","6,071","83,805","1,048","121,825"
5,성남시중원구,"207,205","82,650","26,147","41,203","7,825","6,653","81,828",822,"124,555"
6,성남시분당구,"330,404","164,190","46,797","107,353","4,380","4,789","163,319",871,"166,214"
7,의정부시,"299,556","130,381","41,367","77,107","3,905","6,738","129,117","1,264","169,175"
8,안양시만안구,"204,688","95,595","31,436","53,699","4,500","5,055","94,690",905,"109,093"
9,안양시동안구,"259,321","134,635","46,671","76,522","4,696","5,789","133,678",957,"124,686"


In [91]:
gyeonggi_4th = gyeonggi_4th.assign(
    시도='경기도'
)[['시도'] + gyeonggi_4th.columns.tolist()]

In [92]:
gyeonggi_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진대제,득표수_2_한나라당_김문수,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수
0,경기도,수원시장안구,"206,015","98,094","31,861","56,361","3,366","5,795","97,383",711,"107,921"
1,경기도,수원시권선구,"217,455","97,485","31,707","56,267","2,822","5,838","96,634",851,"119,970"
2,경기도,수원시팔달구,"166,662","72,030","22,480","42,829","2,501","3,520","71,330",700,"94,632"
3,경기도,수원시영통구,"173,454","82,006","31,247","45,246","1,653","3,442","81,588",418,"91,448"
4,경기도,성남시수정구,"206,678","84,853","28,638","41,683","7,413","6,071","83,805","1,048","121,825"


In [93]:
gyeonggi_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               44 non-null     object
 1   구시군              44 non-null     object
 2   선거인수             44 non-null     object
 3   투표수              44 non-null     object
 4   득표수_1_열린우리당_진대제  44 non-null     object
 5   득표수_2_한나라당_김문수   44 non-null     object
 6   득표수_3_민주당_최기복    44 non-null     object
 7   득표수_4_민주노동당_김용한  44 non-null     object
 8   득표수_계            44 non-null     object
 9   무효투표수            44 non-null     object
 10  기권수              44 non-null     object
dtypes: object(11)
memory usage: 3.9+ KB


In [94]:
gyeonggi_4th = gyeonggi_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [95]:
# 수치형 열만 합계 구하기
summary_row = gyeonggi_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경기도')

# summary_row를 맨 위에 붙이기
gyeonggi_4th_with_total = pd.concat([summary_row, gyeonggi_4th], ignore_index=True)

In [96]:
gyeonggi_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진대제,득표수_2_한나라당_김문수,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수
0,경기도,합계,7918828,3695552,1124317,2181677,148409,201106,3655509,40043,4223276
1,경기도,수원시장안구,206015,98094,31861,56361,3366,5795,97383,711,107921
2,경기도,수원시권선구,217455,97485,31707,56267,2822,5838,96634,851,119970
3,경기도,수원시팔달구,166662,72030,22480,42829,2501,3520,71330,700,94632
4,경기도,수원시영통구,173454,82006,31247,45246,1653,3442,81588,418,91448
5,경기도,성남시수정구,206678,84853,28638,41683,7413,6071,83805,1048,121825
6,경기도,성남시중원구,207205,82650,26147,41203,7825,6653,81828,822,124555
7,경기도,성남시분당구,330404,164190,46797,107353,4380,4789,163319,871,166214
8,경기도,의정부시,299556,130381,41367,77107,3905,6738,129117,1264,169175
9,경기도,안양시만안구,204688,95595,31436,53699,4500,5055,94690,905,109093


In [97]:
gyeonggi_4th_with_total.to_csv("temp1_governor_gyeonggi_4.csv", index=False, encoding="utf-8-sig")


## Gangwon


In [98]:
gangwon_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/42_강원"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/42_강원
------------------------------------------------------------
폴더에서 18개 파일 발견

처리 중: 4201_춘천시_강원도.xls (지역: 춘천시)
WARNING *** file size (214280) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 35

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 35행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4202_원주시_강원도.xls (지역: 원주시)
WARNING *** file size (207438) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 34

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 34행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4203_강릉시_강원도.xls (지역: 강릉시)
WARNING *** file size (186652) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 31

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 31행 → 필터링 후: 1행
  ✓ 완료: 1행 추

In [99]:
gangwon_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n이창복,한나라당\n김진선,민주당\n유재규,국민중심당\n유승규,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,춘천시,NaN,합계,"192,644",NaN,"104,177","36,231","45,125","11,660","7,673",NaN,"100,689","3,488","88,467",NaN,NaN,0.0
1,원주시,NaN,합계,"215,770",NaN,"106,902","31,345","70,598","2,397","1,321",NaN,"105,661","1,241","108,868",NaN,NaN,0.0
2,강릉시,NaN,합계,"171,138",NaN,"92,512","12,467","74,805","2,064","1,709",NaN,"91,045","1,467","78,626",NaN,NaN,0.0
3,동해시,NaN,합계,"74,884",NaN,"43,269","4,903","36,395",803,586,NaN,"42,687",582,"31,615",NaN,NaN,0.0
4,삼척시,NaN,합계,"57,751",NaN,"39,784","4,897","31,894",993,"1,174",NaN,"38,958",826,"17,967",NaN,NaN,0.0
5,태백시,NaN,합계,"41,345",NaN,"27,807","4,148","21,005",464,"1,781",NaN,"27,398",409,"13,538",NaN,NaN,0.0
6,정선군,NaN,합계,"35,948",NaN,"25,156","4,982","18,270",715,697,NaN,"24,664",492,"10,792",NaN,NaN,0.0
7,속초시,NaN,합계,"64,750",NaN,"34,125","6,072","26,288",826,471,NaN,"33,657",468,"30,625",NaN,NaN,0.0
8,고성군,NaN,합계,"25,708",NaN,"18,721","3,010","14,514",493,299,NaN,"18,316",405,"6,987",NaN,NaN,0.0
9,양양군,NaN,합계,"23,582",NaN,"16,899","2,584","13,324",415,240,NaN,"16,563",336,"6,683",NaN,NaN,0.0


In [100]:
gangwon_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n이창복',
 '한나라당\n김진선',
 '민주당\n유재규',
 '국민중심당\n유승규',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [101]:
rename_gangwon = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n이창복': '득표수_1_열린우리당_이창복',
    '한나라당\n김진선': '득표수_2_한나라당_김진선',
    '민주당\n유재규': '득표수_3_민주당_유재규',
    '국민중심당\n유승규': '득표수_5_국민중심당_유승규',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [102]:
gangwon_4th = gangwon_4th.rename(columns=rename_gangwon).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
gangwon_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_이창복,득표수_2_한나라당_김진선,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수
0,춘천시,"192,644","104,177","36,231","45,125","11,660","7,673","100,689","3,488","88,467"
1,원주시,"215,770","106,902","31,345","70,598","2,397","1,321","105,661","1,241","108,868"
2,강릉시,"171,138","92,512","12,467","74,805","2,064","1,709","91,045","1,467","78,626"
3,동해시,"74,884","43,269","4,903","36,395",803,586,"42,687",582,"31,615"
4,삼척시,"57,751","39,784","4,897","31,894",993,"1,174","38,958",826,"17,967"
5,태백시,"41,345","27,807","4,148","21,005",464,"1,781","27,398",409,"13,538"
6,정선군,"35,948","25,156","4,982","18,270",715,697,"24,664",492,"10,792"
7,속초시,"64,750","34,125","6,072","26,288",826,471,"33,657",468,"30,625"
8,고성군,"25,708","18,721","3,010","14,514",493,299,"18,316",405,"6,987"
9,양양군,"23,582","16,899","2,584","13,324",415,240,"16,563",336,"6,683"


In [103]:
gangwon_4th = gangwon_4th.assign(
    시도='강원도'
)[['시도'] + gangwon_4th.columns.tolist()]

In [104]:
gangwon_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이창복,득표수_2_한나라당_김진선,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수
0,강원도,춘천시,"192,644","104,177","36,231","45,125","11,660","7,673","100,689","3,488","88,467"
1,강원도,원주시,"215,770","106,902","31,345","70,598","2,397","1,321","105,661","1,241","108,868"
2,강원도,강릉시,"171,138","92,512","12,467","74,805","2,064","1,709","91,045","1,467","78,626"
3,강원도,동해시,"74,884","43,269","4,903","36,395",803,586,"42,687",582,"31,615"
4,강원도,삼척시,"57,751","39,784","4,897","31,894",993,"1,174","38,958",826,"17,967"


In [105]:
gangwon_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               18 non-null     object
 1   구시군              18 non-null     object
 2   선거인수             18 non-null     object
 3   투표수              18 non-null     object
 4   득표수_1_열린우리당_이창복  18 non-null     object
 5   득표수_2_한나라당_김진선   18 non-null     object
 6   득표수_3_민주당_유재규    18 non-null     object
 7   득표수_5_국민중심당_유승규  18 non-null     object
 8   득표수_계            18 non-null     object
 9   무효투표수            18 non-null     object
 10  기권수              18 non-null     object
dtypes: object(11)
memory usage: 1.7+ KB


In [106]:
gangwon_4th = gangwon_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [107]:
# 수치형 열만 합계 구하기
summary_row = gangwon_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '강원도')

# summary_row를 맨 위에 붙이기
gangwon_4th_with_total = pd.concat([summary_row, gangwon_4th], ignore_index=True)

In [108]:
gangwon_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이창복,득표수_2_한나라당_김진선,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수
0,강원도,합계,1160977,681633,148302,471613,29028,19383,668326,13307,479344
1,강원도,춘천시,192644,104177,36231,45125,11660,7673,100689,3488,88467
2,강원도,원주시,215770,106902,31345,70598,2397,1321,105661,1241,108868
3,강원도,강릉시,171138,92512,12467,74805,2064,1709,91045,1467,78626
4,강원도,동해시,74884,43269,4903,36395,803,586,42687,582,31615
5,강원도,삼척시,57751,39784,4897,31894,993,1174,38958,826,17967
6,강원도,태백시,41345,27807,4148,21005,464,1781,27398,409,13538
7,강원도,정선군,35948,25156,4982,18270,715,697,24664,492,10792
8,강원도,속초시,64750,34125,6072,26288,826,471,33657,468,30625
9,강원도,고성군,25708,18721,3010,14514,493,299,18316,405,6987


In [109]:
gangwon_4th_with_total.to_csv("temp1_governor_gangwon_4.csv", index=False, encoding="utf-8-sig")


## Chungbuk


In [110]:
chungbuk_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/43_충북"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/43_충북
------------------------------------------------------------
폴더에서 13개 파일 발견

처리 중: 4301_청주시상당구_충청북도.xls (지역: 청주시상당구)
WARNING *** file size (133126) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 23

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 23행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4302_청주시흥덕구_충청북도.xls (지역: 청주시흥덕구)
WARNING *** file size (153358) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4303_충주시_충청북도.xls (지역: 충주시)
WARNING *** file size (214188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 35

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 35행 → 필터링 후: 

In [111]:
chungbuk_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n한범덕,한나라당\n정우택,민주노동당\n배창호,국민중심당\n조병세,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,청주시상당구,NaN,합계,"188,493",NaN,"89,032","28,260","52,776","5,015","2,114",NaN,"88,165",867,"99,461",NaN,NaN,0.0
1,청주시흥덕구,NaN,합계,"269,249",NaN,"121,820","38,846","70,472","8,623","2,740",NaN,"120,681","1,139","147,429",NaN,NaN,0.0
2,충주시,NaN,합계,"155,769",NaN,"83,466","24,287","50,932","4,746","2,095",NaN,"82,060","1,406","72,303",NaN,NaN,0.0
3,제천시,NaN,합계,"105,920",NaN,"60,200","16,169","37,174","3,981","1,845",NaN,"59,169","1,031","45,720",NaN,NaN,0.0
4,단양군,NaN,합계,"27,458",NaN,"19,533","5,643","11,497",994,883,NaN,"19,017",516,"7,925",NaN,NaN,0.0
5,청원군,NaN,합계,"95,259",NaN,"53,191","18,015","27,833","3,274","2,962",NaN,"52,084","1,107","42,068",NaN,NaN,0.0
6,영동군,NaN,합계,"41,700",NaN,"29,569","9,031","15,383","2,646","1,708",NaN,"28,768",801,"12,131",NaN,NaN,0.0
7,보은군,NaN,합계,"30,273",NaN,"22,890","8,833","11,107","1,282","1,078",NaN,"22,300",590,"7,383",NaN,NaN,0.0
8,옥천군,NaN,합계,"44,454",NaN,"31,353","11,522","15,423","2,014","1,362",NaN,"30,321","1,032","13,101",NaN,NaN,0.0
9,음성군,NaN,합계,"66,010",NaN,"38,457","8,678","25,794","2,299","1,030",NaN,"37,801",656,"27,553",NaN,NaN,0.0


In [112]:
chungbuk_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n한범덕',
 '한나라당\n정우택',
 '민주노동당\n배창호',
 '국민중심당\n조병세',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [113]:
rename_chungbuk = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n한범덕': '득표수_1_열린우리당_한범덕',
    '한나라당\n정우택': '득표수_2_한나라당_정우택',
    '민주노동당\n배창호': '득표수_4_민주노동당_배창호',
    '국민중심당\n조병세': '득표수_5_국민중심당_조병세',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [114]:
chungbuk_4th = chungbuk_4th.rename(columns=rename_chungbuk).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
chungbuk_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_한범덕,득표수_2_한나라당_정우택,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수
0,청주시상당구,"188,493","89,032","28,260","52,776","5,015","2,114","88,165",867,"99,461"
1,청주시흥덕구,"269,249","121,820","38,846","70,472","8,623","2,740","120,681","1,139","147,429"
2,충주시,"155,769","83,466","24,287","50,932","4,746","2,095","82,060","1,406","72,303"
3,제천시,"105,920","60,200","16,169","37,174","3,981","1,845","59,169","1,031","45,720"
4,단양군,"27,458","19,533","5,643","11,497",994,883,"19,017",516,"7,925"
5,청원군,"95,259","53,191","18,015","27,833","3,274","2,962","52,084","1,107","42,068"
6,영동군,"41,700","29,569","9,031","15,383","2,646","1,708","28,768",801,"12,131"
7,보은군,"30,273","22,890","8,833","11,107","1,282","1,078","22,300",590,"7,383"
8,옥천군,"44,454","31,353","11,522","15,423","2,014","1,362","30,321","1,032","13,101"
9,음성군,"66,010","38,457","8,678","25,794","2,299","1,030","37,801",656,"27,553"


In [115]:
chungbuk_4th = chungbuk_4th.assign(
    시도='충청북도'
)[['시도'] + chungbuk_4th.columns.tolist()]

In [116]:
chungbuk_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_한범덕,득표수_2_한나라당_정우택,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수
0,충청북도,청주시상당구,"188,493","89,032","28,260","52,776","5,015","2,114","88,165",867,"99,461"
1,충청북도,청주시흥덕구,"269,249","121,820","38,846","70,472","8,623","2,740","120,681","1,139","147,429"
2,충청북도,충주시,"155,769","83,466","24,287","50,932","4,746","2,095","82,060","1,406","72,303"
3,충청북도,제천시,"105,920","60,200","16,169","37,174","3,981","1,845","59,169","1,031","45,720"
4,충청북도,단양군,"27,458","19,533","5,643","11,497",994,883,"19,017",516,"7,925"


In [117]:
chungbuk_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               13 non-null     object
 1   구시군              13 non-null     object
 2   선거인수             13 non-null     object
 3   투표수              13 non-null     object
 4   득표수_1_열린우리당_한범덕  13 non-null     object
 5   득표수_2_한나라당_정우택   13 non-null     object
 6   득표수_4_민주노동당_배창호  13 non-null     object
 7   득표수_5_국민중심당_조병세  13 non-null     object
 8   득표수_계            13 non-null     object
 9   무효투표수            13 non-null     object
 10  기권수              13 non-null     object
dtypes: object(11)
memory usage: 1.2+ KB


In [118]:
chungbuk_4th = chungbuk_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [119]:
# 수치형 열만 합계 구하기
summary_row = chungbuk_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청북도')

# summary_row를 맨 위에 붙이기
chungbuk_4th_with_total = pd.concat([summary_row, chungbuk_4th], ignore_index=True)

In [120]:
chungbuk_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_한범덕,득표수_2_한나라당_정우택,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수
0,충청북도,합계,1126282,616053,185426,361157,39095,19646,605324,10729,510229
1,충청북도,청주시상당구,188493,89032,28260,52776,5015,2114,88165,867,99461
2,충청북도,청주시흥덕구,269249,121820,38846,70472,8623,2740,120681,1139,147429
3,충청북도,충주시,155769,83466,24287,50932,4746,2095,82060,1406,72303
4,충청북도,제천시,105920,60200,16169,37174,3981,1845,59169,1031,45720
5,충청북도,단양군,27458,19533,5643,11497,994,883,19017,516,7925
6,충청북도,청원군,95259,53191,18015,27833,3274,2962,52084,1107,42068
7,충청북도,영동군,41700,29569,9031,15383,2646,1708,28768,801,12131
8,충청북도,보은군,30273,22890,8833,11107,1282,1078,22300,590,7383
9,충청북도,옥천군,44454,31353,11522,15423,2014,1362,30321,1032,13101


In [121]:
chungbuk_4th_with_total.to_csv("temp1_governor_chungbuk_4.csv", index=False, encoding="utf-8-sig")


## Chungnam


In [122]:
chungnam_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/44_충남"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/44_충남
------------------------------------------------------------
폴더에서 16개 파일 발견

처리 중: 4401_천안시_충청남도.xls (지역: 천안시)
WARNING *** file size (236691) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 44

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 44행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4402_공주시_충청남도.xls (지역: 공주시)
WARNING *** file size (159887) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 27

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 27행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4403_보령시_충청남도.xls (지역: 보령시)
WARNING *** file size (153166) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1

In [123]:
chungnam_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n오영교,한나라당\n이완구,민주노동당\n이용길,국민중심당\n이명수,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,천안시,NaN,합계,"372,195",NaN,"162,726","36,426","84,436","15,778","24,136",NaN,"160,776","1,950","209,469",NaN,NaN,0.0
1,공주시,NaN,합계,"100,960",NaN,"61,082","13,868","21,948","3,101","20,682",NaN,"59,599","1,483","39,878",NaN,NaN,0.0
2,보령시,NaN,합계,"84,236",NaN,"52,539","16,826","20,818","2,392","11,282",NaN,"51,318","1,221","31,697",NaN,NaN,0.0
3,아산시,NaN,합계,"156,817",NaN,"74,771","10,839","28,561","6,590","27,467",NaN,"73,457","1,314","82,046",NaN,NaN,0.0
4,서산시,NaN,합계,"112,204",NaN,"61,310","13,144","27,317","3,995","15,341",NaN,"59,797","1,513","50,894",NaN,NaN,0.0
5,태안군,NaN,합계,"51,770",NaN,"35,084","7,572","13,756","1,814","10,701",NaN,"33,843","1,241","16,686",NaN,NaN,0.0
6,금산군,NaN,합계,"47,627",NaN,"32,263","6,017","12,045","1,531","11,653",NaN,"31,246","1,017","15,364",NaN,NaN,0.0
7,연기군,NaN,합계,"65,548",NaN,"39,859","13,698","12,927","1,777","10,431",NaN,"38,833","1,026","25,689",NaN,NaN,0.0
8,논산시,NaN,합계,"104,305",NaN,"57,835","13,044","20,497","2,926","19,699",NaN,"56,166","1,669","46,470",NaN,NaN,0.0
9,계룡시,NaN,합계,"24,442",NaN,"15,359","3,526","7,846",561,"3,209",NaN,"15,142",217,"9,083",NaN,NaN,0.0


In [124]:
chungnam_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n오영교',
 '한나라당\n이완구',
 '민주노동당\n이용길',
 '국민중심당\n이명수',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [125]:
rename_chungnam = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n오영교': '득표수_1_열린우리당_오영교',
    '한나라당\n이완구': '득표수_2_한나라당_이완구',
    '민주노동당\n이용길': '득표수_4_민주노동당_이용길',
    '국민중심당\n이명수': '득표수_5_국민중심당_이명수',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [126]:
chungnam_4th = chungnam_4th.rename(columns=rename_chungnam).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
chungnam_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_오영교,득표수_2_한나라당_이완구,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수
0,천안시,"372,195","162,726","36,426","84,436","15,778","24,136","160,776","1,950","209,469"
1,공주시,"100,960","61,082","13,868","21,948","3,101","20,682","59,599","1,483","39,878"
2,보령시,"84,236","52,539","16,826","20,818","2,392","11,282","51,318","1,221","31,697"
3,아산시,"156,817","74,771","10,839","28,561","6,590","27,467","73,457","1,314","82,046"
4,서산시,"112,204","61,310","13,144","27,317","3,995","15,341","59,797","1,513","50,894"
5,태안군,"51,770","35,084","7,572","13,756","1,814","10,701","33,843","1,241","16,686"
6,금산군,"47,627","32,263","6,017","12,045","1,531","11,653","31,246","1,017","15,364"
7,연기군,"65,548","39,859","13,698","12,927","1,777","10,431","38,833","1,026","25,689"
8,논산시,"104,305","57,835","13,044","20,497","2,926","19,699","56,166","1,669","46,470"
9,계룡시,"24,442","15,359","3,526","7,846",561,"3,209","15,142",217,"9,083"


In [127]:
chungnam_4th = chungnam_4th.assign(
    시도='충청남도'
)[['시도'] + chungnam_4th.columns.tolist()]

In [128]:
chungnam_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오영교,득표수_2_한나라당_이완구,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수
0,충청남도,천안시,"372,195","162,726","36,426","84,436","15,778","24,136","160,776","1,950","209,469"
1,충청남도,공주시,"100,960","61,082","13,868","21,948","3,101","20,682","59,599","1,483","39,878"
2,충청남도,보령시,"84,236","52,539","16,826","20,818","2,392","11,282","51,318","1,221","31,697"
3,충청남도,아산시,"156,817","74,771","10,839","28,561","6,590","27,467","73,457","1,314","82,046"
4,충청남도,서산시,"112,204","61,310","13,144","27,317","3,995","15,341","59,797","1,513","50,894"


In [129]:
chungnam_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               16 non-null     object
 1   구시군              16 non-null     object
 2   선거인수             16 non-null     object
 3   투표수              16 non-null     object
 4   득표수_1_열린우리당_오영교  16 non-null     object
 5   득표수_2_한나라당_이완구   16 non-null     object
 6   득표수_4_민주노동당_이용길  16 non-null     object
 7   득표수_5_국민중심당_이명수  16 non-null     object
 8   득표수_계            16 non-null     object
 9   무효투표수            16 non-null     object
 10  기권수              16 non-null     object
dtypes: object(11)
memory usage: 1.5+ KB


In [130]:
chungnam_4th = chungnam_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [131]:
# 수치형 열만 합계 구하기
summary_row = chungnam_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청남도')

# summary_row를 맨 위에 붙이기
chungnam_4th_with_total = pd.concat([summary_row, chungnam_4th], ignore_index=True)

In [132]:
chungnam_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오영교,득표수_2_한나라당_이완구,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수
0,충청남도,합계,1503240,838462,178169,379420,52417,209254,819260,19202,664778
1,충청남도,천안시,372195,162726,36426,84436,15778,24136,160776,1950,209469
2,충청남도,공주시,100960,61082,13868,21948,3101,20682,59599,1483,39878
3,충청남도,보령시,84236,52539,16826,20818,2392,11282,51318,1221,31697
4,충청남도,아산시,156817,74771,10839,28561,6590,27467,73457,1314,82046
5,충청남도,서산시,112204,61310,13144,27317,3995,15341,59797,1513,50894
6,충청남도,태안군,51770,35084,7572,13756,1814,10701,33843,1241,16686
7,충청남도,금산군,47627,32263,6017,12045,1531,11653,31246,1017,15364
8,충청남도,연기군,65548,39859,13698,12927,1777,10431,38833,1026,25689
9,충청남도,논산시,104305,57835,13044,20497,2926,19699,56166,1669,46470


In [133]:
chungnam_4th_with_total.to_csv("temp1_governor_chungnam_4.csv", index=False, encoding="utf-8-sig")


## Jeonbuk


In [134]:
jeonbuk_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/45_전북"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/45_전북
------------------------------------------------------------
폴더에서 15개 파일 발견

처리 중: 4501_전주시완산구_전라북도.xls (지역: 전주시완산구)
WARNING *** file size (166803) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 28

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 28행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4502_전주시덕진구_전라북도.xls (지역: 전주시덕진구)
WARNING *** file size (146565) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 25

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 25행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4503_군산시_전라북도.xls (지역: 군산시)
WARNING *** file size (256697) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 47

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 47행 → 필터링 후: 

In [135]:
jeonbuk_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n김완주,한나라당\n문용주,민주당\n정균환,민주노동당\n염경석,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,전주시완산구,NaN,합계,"252,082",NaN,"121,291","59,401","6,322","45,973","8,406",NaN,"120,102","1,189","130,791",NaN,NaN,0.0
1,전주시덕진구,NaN,합계,"198,612",NaN,"94,137","45,562","5,075","34,130","8,560",NaN,"93,327",810,"104,475",NaN,NaN,0.0
2,군산시,NaN,합계,"198,467",NaN,"111,760","44,385","19,593","36,122","9,086",NaN,"109,186","2,574","86,707",NaN,NaN,0.0
3,익산시,NaN,합계,"235,847",NaN,"123,052","55,008","8,189","48,019","9,442",NaN,"120,658","2,394","112,795",NaN,NaN,0.0
4,정읍시,NaN,합계,"99,727",NaN,"62,082","30,801","3,181","21,387","5,254",NaN,"60,623","1,459","37,645",NaN,NaN,0.0
5,남원시,NaN,합계,"72,197",NaN,"50,215","27,923","2,944","15,868","2,271",NaN,"49,006","1,209","21,982",NaN,NaN,0.0
6,김제시,NaN,합계,"82,928",NaN,"55,599","27,773","3,576","19,322","3,272",NaN,"53,943","1,656","27,329",NaN,NaN,0.0
7,완주군,NaN,합계,"66,720",NaN,"41,559","19,070","2,019","15,435","4,089",NaN,"40,613",946,"25,161",NaN,NaN,0.0
8,진안군,NaN,합계,"23,432",NaN,"17,528","9,415","1,437","4,987","1,186",NaN,"17,025",503,"5,904",NaN,NaN,0.0
9,무주군,NaN,합계,"21,301",NaN,"16,835","8,558","1,622","4,857","1,203",NaN,"16,240",595,"4,466",NaN,NaN,0.0


In [136]:
jeonbuk_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n김완주',
 '한나라당\n문용주',
 '민주당\n정균환',
 '민주노동당\n염경석',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [137]:
rename_jeonbuk = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n김완주': '득표수_1_열린우리당_김완주',
    '한나라당\n문용주': '득표수_2_한나라당_문용주',
    '민주당\n정균환': '득표수_3_민주당_정균환',
    '민주노동당\n염경석': '득표수_4_민주노동당_염경석',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [138]:
jeonbuk_4th = jeonbuk_4th.rename(columns=rename_jeonbuk).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
jeonbuk_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_김완주,득표수_2_한나라당_문용주,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수
0,전주시완산구,"252,082","121,291","59,401","6,322","45,973","8,406","120,102","1,189","130,791"
1,전주시덕진구,"198,612","94,137","45,562","5,075","34,130","8,560","93,327",810,"104,475"
2,군산시,"198,467","111,760","44,385","19,593","36,122","9,086","109,186","2,574","86,707"
3,익산시,"235,847","123,052","55,008","8,189","48,019","9,442","120,658","2,394","112,795"
4,정읍시,"99,727","62,082","30,801","3,181","21,387","5,254","60,623","1,459","37,645"
5,남원시,"72,197","50,215","27,923","2,944","15,868","2,271","49,006","1,209","21,982"
6,김제시,"82,928","55,599","27,773","3,576","19,322","3,272","53,943","1,656","27,329"
7,완주군,"66,720","41,559","19,070","2,019","15,435","4,089","40,613",946,"25,161"
8,진안군,"23,432","17,528","9,415","1,437","4,987","1,186","17,025",503,"5,904"
9,무주군,"21,301","16,835","8,558","1,622","4,857","1,203","16,240",595,"4,466"


In [139]:
jeonbuk_4th = jeonbuk_4th.assign(
    시도='전라북도'
)[['시도'] + jeonbuk_4th.columns.tolist()]

In [140]:
jeonbuk_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김완주,득표수_2_한나라당_문용주,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수
0,전라북도,전주시완산구,"252,082","121,291","59,401","6,322","45,973","8,406","120,102","1,189","130,791"
1,전라북도,전주시덕진구,"198,612","94,137","45,562","5,075","34,130","8,560","93,327",810,"104,475"
2,전라북도,군산시,"198,467","111,760","44,385","19,593","36,122","9,086","109,186","2,574","86,707"
3,전라북도,익산시,"235,847","123,052","55,008","8,189","48,019","9,442","120,658","2,394","112,795"
4,전라북도,정읍시,"99,727","62,082","30,801","3,181","21,387","5,254","60,623","1,459","37,645"


In [141]:
jeonbuk_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               15 non-null     object
 1   구시군              15 non-null     object
 2   선거인수             15 non-null     object
 3   투표수              15 non-null     object
 4   득표수_1_열린우리당_김완주  15 non-null     object
 5   득표수_2_한나라당_문용주   15 non-null     object
 6   득표수_3_민주당_정균환    15 non-null     object
 7   득표수_4_민주노동당_염경석  15 non-null     object
 8   득표수_계            15 non-null     object
 9   무효투표수            15 non-null     object
 10  기권수              15 non-null     object
dtypes: object(11)
memory usage: 1.4+ KB


In [142]:
jeonbuk_4th = jeonbuk_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [143]:
# 수치형 열만 합계 구하기
summary_row = jeonbuk_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라북도')

# summary_row를 맨 위에 붙이기
jeonbuk_4th_with_total = pd.concat([summary_row, jeonbuk_4th], ignore_index=True)

In [144]:
jeonbuk_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김완주,득표수_2_한나라당_문용주,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수
0,전라북도,합계,1429632,827387,389436,62922,295891,61672,809921,17466,602245
1,전라북도,전주시완산구,252082,121291,59401,6322,45973,8406,120102,1189,130791
2,전라북도,전주시덕진구,198612,94137,45562,5075,34130,8560,93327,810,104475
3,전라북도,군산시,198467,111760,44385,19593,36122,9086,109186,2574,86707
4,전라북도,익산시,235847,123052,55008,8189,48019,9442,120658,2394,112795
5,전라북도,정읍시,99727,62082,30801,3181,21387,5254,60623,1459,37645
6,전라북도,남원시,72197,50215,27923,2944,15868,2271,49006,1209,21982
7,전라북도,김제시,82928,55599,27773,3576,19322,3272,53943,1656,27329
8,전라북도,완주군,66720,41559,19070,2019,15435,4089,40613,946,25161
9,전라북도,진안군,23432,17528,9415,1437,4987,1186,17025,503,5904


In [145]:
jeonbuk_4th_with_total.to_csv("temp1_governor_jeonbuk_4.csv", index=False, encoding="utf-8-sig")


## Jeonnam


In [146]:
jeonnam_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/46_전남"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/46_전남
------------------------------------------------------------
폴더에서 22개 파일 발견

처리 중: 4601_목포시_전라남도.xls (지역: 목포시)
WARNING *** file size (214218) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 35

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 35행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4602_여수시_전라남도.xls (지역: 여수시)
WARNING *** file size (236494) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 44

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 44행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4604_순천시_전라남도.xls (지역: 순천시)
WARNING *** file size (207500) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 34

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 34행 → 필터링 후: 1행
  ✓ 완료: 1

In [147]:
jeonnam_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n서범석,한나라당\n박재순,민주당\n박준영,민주노동당\n박웅두,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,목포시,NaN,합계,"176,932",NaN,"88,358","12,717","2,667","66,179","5,513",NaN,"87,076","1,282","88,574",NaN,NaN,0.0
1,여수시,NaN,합계,"224,001",NaN,"122,812","32,665","4,973","76,454","6,702",NaN,"120,794","2,018","101,189",NaN,NaN,0.0
2,순천시,NaN,합계,"194,227",NaN,"105,389","27,461","4,311","63,894","7,802",NaN,"103,468","1,921","88,838",NaN,NaN,0.0
3,나주시,NaN,합계,"79,340",NaN,"55,195","5,955","2,484","40,865","4,087",NaN,"53,391","1,804","24,145",NaN,NaN,0.0
4,광양시,NaN,합계,"96,951",NaN,"58,745","20,925","4,266","27,503","5,136",NaN,"57,830",915,"38,206",NaN,NaN,0.0
5,담양군,NaN,합계,"41,974",NaN,"29,475","5,756","1,643","19,625","1,621",NaN,"28,645",830,"12,499",NaN,NaN,0.0
6,장성군,NaN,합계,"39,626",NaN,"27,752","3,757","1,948","19,383","1,938",NaN,"27,026",726,"11,874",NaN,NaN,0.0
7,곡성군,NaN,합계,"28,138",NaN,"21,815","4,152","1,439","12,768","2,730",NaN,"21,089",726,"6,323",NaN,NaN,0.0
8,구례군,NaN,합계,"24,281",NaN,"19,123","4,412","1,118","11,643","1,396",NaN,"18,569",554,"5,158",NaN,NaN,0.0
9,고흥군,NaN,합계,"69,751",NaN,"52,672","8,994","2,733","36,221","2,755",NaN,"50,703","1,969","17,079",NaN,NaN,0.0


In [148]:
jeonnam_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n서범석',
 '한나라당\n박재순',
 '민주당\n박준영',
 '민주노동당\n박웅두',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [149]:
rename_jeonnam = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n서범석': '득표수_1_열린우리당_서범석',
    '한나라당\n박재순': '득표수_2_한나라당_박재순',
    '민주당\n박준영': '득표수_3_민주당_박준영',
    '민주노동당\n박웅두': '득표수_4_민주노동당_박웅두',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [150]:
jeonnam_4th = jeonnam_4th.rename(columns=rename_jeonnam).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
jeonnam_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_서범석,득표수_2_한나라당_박재순,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수
0,목포시,"176,932","88,358","12,717","2,667","66,179","5,513","87,076","1,282","88,574"
1,여수시,"224,001","122,812","32,665","4,973","76,454","6,702","120,794","2,018","101,189"
2,순천시,"194,227","105,389","27,461","4,311","63,894","7,802","103,468","1,921","88,838"
3,나주시,"79,340","55,195","5,955","2,484","40,865","4,087","53,391","1,804","24,145"
4,광양시,"96,951","58,745","20,925","4,266","27,503","5,136","57,830",915,"38,206"
5,담양군,"41,974","29,475","5,756","1,643","19,625","1,621","28,645",830,"12,499"
6,장성군,"39,626","27,752","3,757","1,948","19,383","1,938","27,026",726,"11,874"
7,곡성군,"28,138","21,815","4,152","1,439","12,768","2,730","21,089",726,"6,323"
8,구례군,"24,281","19,123","4,412","1,118","11,643","1,396","18,569",554,"5,158"
9,고흥군,"69,751","52,672","8,994","2,733","36,221","2,755","50,703","1,969","17,079"


In [151]:
jeonnam_4th = jeonnam_4th.assign(
    시도='전라남도'
)[['시도'] + jeonnam_4th.columns.tolist()]

In [152]:
jeonnam_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_서범석,득표수_2_한나라당_박재순,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수
0,전라남도,목포시,"176,932","88,358","12,717","2,667","66,179","5,513","87,076","1,282","88,574"
1,전라남도,여수시,"224,001","122,812","32,665","4,973","76,454","6,702","120,794","2,018","101,189"
2,전라남도,순천시,"194,227","105,389","27,461","4,311","63,894","7,802","103,468","1,921","88,838"
3,전라남도,나주시,"79,340","55,195","5,955","2,484","40,865","4,087","53,391","1,804","24,145"
4,전라남도,광양시,"96,951","58,745","20,925","4,266","27,503","5,136","57,830",915,"38,206"


In [153]:
jeonnam_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               22 non-null     object
 1   구시군              22 non-null     object
 2   선거인수             22 non-null     object
 3   투표수              22 non-null     object
 4   득표수_1_열린우리당_서범석  22 non-null     object
 5   득표수_2_한나라당_박재순   22 non-null     object
 6   득표수_3_민주당_박준영    22 non-null     object
 7   득표수_4_민주노동당_박웅두  22 non-null     object
 8   득표수_계            22 non-null     object
 9   무효투표수            22 non-null     object
 10  기권수              22 non-null     object
dtypes: object(11)
memory usage: 2.0+ KB


In [154]:
jeonnam_4th = jeonnam_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [155]:
# 수치형 열만 합계 구하기
summary_row = jeonnam_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '전라남도')

# summary_row를 맨 위에 붙이기
jeonnam_4th_with_total = pd.concat([summary_row, jeonnam_4th], ignore_index=True)

In [156]:
jeonnam_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_서범석,득표수_2_한나라당_박재순,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수
0,전라남도,합계,1513912,973820,181756,55444,640894,68702,946796,27024,540092
1,전라남도,목포시,176932,88358,12717,2667,66179,5513,87076,1282,88574
2,전라남도,여수시,224001,122812,32665,4973,76454,6702,120794,2018,101189
3,전라남도,순천시,194227,105389,27461,4311,63894,7802,103468,1921,88838
4,전라남도,나주시,79340,55195,5955,2484,40865,4087,53391,1804,24145
5,전라남도,광양시,96951,58745,20925,4266,27503,5136,57830,915,38206
6,전라남도,담양군,41974,29475,5756,1643,19625,1621,28645,830,12499
7,전라남도,장성군,39626,27752,3757,1948,19383,1938,27026,726,11874
8,전라남도,곡성군,28138,21815,4152,1439,12768,2730,21089,726,6323
9,전라남도,구례군,24281,19123,4412,1118,11643,1396,18569,554,5158


In [157]:
jeonnam_4th_with_total.to_csv("temp1_governor_jeonnam_4.csv", index=False, encoding="utf-8-sig")


## Gyeongbuk


In [158]:
gyeongbuk_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/47_경북"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/47_경북
------------------------------------------------------------
폴더에서 24개 파일 발견

처리 중: 4701_포항시북구_경상북도.xls (지역: 포항시북구)
WARNING *** file size (231966) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 14
데이터 행 수: 27

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 27행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4702_포항시남구_경상북도.xls (지역: 포항시남구)
WARNING *** file size (222131) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 14
데이터 행 수: 26

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 26행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4703_울릉군_경상북도.xls (지역: 울릉군)
WARNING *** file size (70000) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 14
데이터 행 수: 5

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 5행 → 필터링 후: 1행
  ✓ 

In [159]:
gyeongbuk_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n박명재,한나라당\n김관용,column_7,계,무효\n투표수,기권수,column_11,column_12,column_13
0,포항시북구,NaN,합계,"190,449",NaN,"105,458","25,052","78,490",NaN,"103,542","1,916","84,991",NaN,NaN,0.0
1,포항시남구,NaN,합계,"191,369",NaN,"106,552","28,285","76,190",NaN,"104,475","2,077","84,817",NaN,NaN,0.0
2,울릉군,NaN,합계,"8,429",NaN,"7,197","1,978","5,034",NaN,"7,012",185,"1,232",NaN,NaN,0.0
3,경주시,NaN,합계,"210,980",NaN,"126,368","24,020","99,188",NaN,"123,208","3,160","84,612",NaN,NaN,0.0
4,김천시,NaN,합계,"110,274",NaN,"75,345","13,759","59,700",NaN,"73,459","1,886","34,929",NaN,NaN,0.0
5,안동시,NaN,합계,"134,448",NaN,"86,104","20,188","63,435",NaN,"83,623","2,481","48,344",NaN,NaN,0.0
6,구미시,NaN,합계,"269,226",NaN,"132,377","31,617","98,851",NaN,"130,468","1,909","136,849",NaN,NaN,0.0
7,영주시,NaN,합계,"93,247",NaN,"63,459","13,339","48,518",NaN,"61,857","1,602","29,788",NaN,NaN,0.0
8,영천시,NaN,합계,"86,235",NaN,"55,405","15,816","37,956",NaN,"53,772","1,633","30,830",NaN,NaN,0.0
9,상주시,NaN,합계,"88,983",NaN,"63,161","13,414","47,741",NaN,"61,155","2,006","25,822",NaN,NaN,0.0


In [160]:
gyeongbuk_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n박명재',
 '한나라당\n김관용',
 'column_7',
 '계',
 '무효\n투표수',
 '기권수',
 'column_11',
 'column_12',
 'column_13']

In [161]:
rename_gyeongbuk = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n박명재': '득표수_1_열린우리당_박명재',
    '한나라당\n김관용': '득표수_2_한나라당_김관용',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [162]:
gyeongbuk_4th = gyeongbuk_4th.rename(columns=rename_gyeongbuk).drop(columns=['column_0', '읍면동명', 'column_3', 'column_7', 'column_11', 'column_12', 'column_13'])
gyeongbuk_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_박명재,득표수_2_한나라당_김관용,득표수_계,무효투표수,기권수
0,포항시북구,"190,449","105,458","25,052","78,490","103,542","1,916","84,991"
1,포항시남구,"191,369","106,552","28,285","76,190","104,475","2,077","84,817"
2,울릉군,"8,429","7,197","1,978","5,034","7,012",185,"1,232"
3,경주시,"210,980","126,368","24,020","99,188","123,208","3,160","84,612"
4,김천시,"110,274","75,345","13,759","59,700","73,459","1,886","34,929"
5,안동시,"134,448","86,104","20,188","63,435","83,623","2,481","48,344"
6,구미시,"269,226","132,377","31,617","98,851","130,468","1,909","136,849"
7,영주시,"93,247","63,459","13,339","48,518","61,857","1,602","29,788"
8,영천시,"86,235","55,405","15,816","37,956","53,772","1,633","30,830"
9,상주시,"88,983","63,161","13,414","47,741","61,155","2,006","25,822"


In [163]:
gyeongbuk_4th = gyeongbuk_4th.assign(
    시도='경상북도'
)[['시도'] + gyeongbuk_4th.columns.tolist()]

In [164]:
gyeongbuk_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_박명재,득표수_2_한나라당_김관용,득표수_계,무효투표수,기권수
0,경상북도,포항시북구,"190,449","105,458","25,052","78,490","103,542","1,916","84,991"
1,경상북도,포항시남구,"191,369","106,552","28,285","76,190","104,475","2,077","84,817"
2,경상북도,울릉군,"8,429","7,197","1,978","5,034","7,012",185,"1,232"
3,경상북도,경주시,"210,980","126,368","24,020","99,188","123,208","3,160","84,612"
4,경상북도,김천시,"110,274","75,345","13,759","59,700","73,459","1,886","34,929"


In [165]:
gyeongbuk_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               24 non-null     object
 1   구시군              24 non-null     object
 2   선거인수             24 non-null     object
 3   투표수              24 non-null     object
 4   득표수_1_열린우리당_박명재  24 non-null     object
 5   득표수_2_한나라당_김관용   24 non-null     object
 6   득표수_계            24 non-null     object
 7   무효투표수            24 non-null     object
 8   기권수              24 non-null     object
dtypes: object(9)
memory usage: 1.8+ KB


In [166]:
gyeongbuk_4th = gyeongbuk_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [167]:
# 수치형 열만 합계 구하기
summary_row = gyeongbuk_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상북도')

# summary_row를 맨 위에 붙이기
gyeongbuk_4th_with_total = pd.concat([summary_row, gyeongbuk_4th], ignore_index=True)

In [168]:
gyeongbuk_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_박명재,득표수_2_한나라당_김관용,득표수_계,무효투표수,기권수
0,경상북도,합계,2087709,1284342,290358,961363,1251721,32621,803367
1,경상북도,포항시북구,190449,105458,25052,78490,103542,1916,84991
2,경상북도,포항시남구,191369,106552,28285,76190,104475,2077,84817
3,경상북도,울릉군,8429,7197,1978,5034,7012,185,1232
4,경상북도,경주시,210980,126368,24020,99188,123208,3160,84612
5,경상북도,김천시,110274,75345,13759,59700,73459,1886,34929
6,경상북도,안동시,134448,86104,20188,63435,83623,2481,48344
7,경상북도,구미시,269226,132377,31617,98851,130468,1909,136849
8,경상북도,영주시,93247,63459,13339,48518,61857,1602,29788
9,경상북도,영천시,86235,55405,15816,37956,53772,1633,30830


In [169]:
gyeongbuk_4th_with_total.to_csv("temp1_governor_gyeongbuk_4.csv", index=False, encoding="utf-8-sig")


## Gyeongnam


In [170]:
gyeongnam_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/48_경남"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/48_경남
------------------------------------------------------------
폴더에서 20개 파일 발견

처리 중: 4801_창원시_경상남도.xls (지역: 창원시)
WARNING *** file size (139944) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 24

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 24행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4802_마산시_경상남도.xls (지역: 마산시)
WARNING *** file size (270610) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 49

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 49행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4803_진주시_경상남도.xls (지역: 진주시)
WARNING *** file size (304105) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 16
데이터 행 수: 54

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 54행 → 필터링 후: 1행
  ✓ 완료: 1

In [171]:
gyeongnam_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n김두관,한나라당\n김태호,민주노동당\n문성현,국민중심당\n김재주,column_9,계,무효\n투표수,기권수,column_13,column_14,column_15
0,창원시,NaN,합계,"356,493",NaN,"184,115","38,627","112,369","30,193","1,431",NaN,"182,620","1,495","172,378",NaN,NaN,0.0
1,마산시,NaN,합계,"326,663",NaN,"171,791","31,539","122,814","14,066","1,476",NaN,"169,895","1,896","154,872",NaN,NaN,0.0
2,진주시,NaN,합계,"252,537",NaN,"145,232","50,737","77,772","12,912","1,691",NaN,"143,112","2,120","107,305",NaN,NaN,0.0
3,진해시,NaN,합계,"118,111",NaN,"70,223","15,379","46,643","6,344",751,NaN,"69,117","1,106","47,888",NaN,NaN,0.0
4,통영시,NaN,합계,"102,469",NaN,"60,728","13,684","41,786","3,423",827,NaN,"59,720","1,008","41,741",NaN,NaN,0.0
5,고성군,NaN,합계,"46,167",NaN,"33,469","7,661","21,513","2,686",754,NaN,"32,614",855,"12,698",NaN,NaN,0.0
6,사천시,NaN,합계,"86,136",NaN,"57,392","15,408","35,061","4,718",962,NaN,"56,149","1,243","28,744",NaN,NaN,0.0
7,김해시,NaN,합계,"315,263",NaN,"150,966","47,354","87,632","12,929","1,213",NaN,"149,128","1,838","164,297",NaN,NaN,0.0
8,밀양시,NaN,합계,"90,362",NaN,"58,412","15,879","36,650","3,844",825,NaN,"57,198","1,214","31,950",NaN,NaN,0.0
9,거제시,NaN,합계,"142,095",NaN,"75,632","17,024","42,568","14,055",765,NaN,"74,412","1,220","66,463",NaN,NaN,0.0


In [172]:
gyeongnam_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n김두관',
 '한나라당\n김태호',
 '민주노동당\n문성현',
 '국민중심당\n김재주',
 'column_9',
 '계',
 '무효\n투표수',
 '기권수',
 'column_13',
 'column_14',
 'column_15']

In [173]:
rename_gyeongnam = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n김두관': '득표수_1_열린우리당_김두관',
    '한나라당\n김태호': '득표수_2_한나라당_김태호',
    '민주노동당\n문성현': '득표수_4_민주노동당_문성현',
    '국민중심당\n김재주': '득표수_5_국민중심당_김재주',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [174]:
gyeongnam_4th = gyeongnam_4th.rename(columns=rename_gyeongnam).drop(columns=['column_0', '읍면동명', 'column_3', 'column_9', 'column_13', 'column_14', 'column_15'])
gyeongnam_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_김두관,득표수_2_한나라당_김태호,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수
0,창원시,"356,493","184,115","38,627","112,369","30,193","1,431","182,620","1,495","172,378"
1,마산시,"326,663","171,791","31,539","122,814","14,066","1,476","169,895","1,896","154,872"
2,진주시,"252,537","145,232","50,737","77,772","12,912","1,691","143,112","2,120","107,305"
3,진해시,"118,111","70,223","15,379","46,643","6,344",751,"69,117","1,106","47,888"
4,통영시,"102,469","60,728","13,684","41,786","3,423",827,"59,720","1,008","41,741"
5,고성군,"46,167","33,469","7,661","21,513","2,686",754,"32,614",855,"12,698"
6,사천시,"86,136","57,392","15,408","35,061","4,718",962,"56,149","1,243","28,744"
7,김해시,"315,263","150,966","47,354","87,632","12,929","1,213","149,128","1,838","164,297"
8,밀양시,"90,362","58,412","15,879","36,650","3,844",825,"57,198","1,214","31,950"
9,거제시,"142,095","75,632","17,024","42,568","14,055",765,"74,412","1,220","66,463"


In [175]:
gyeongnam_4th = gyeongnam_4th.assign(
    시도='경상남도'
)[['시도'] + gyeongnam_4th.columns.tolist()]

In [176]:
gyeongnam_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김두관,득표수_2_한나라당_김태호,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수
0,경상남도,창원시,"356,493","184,115","38,627","112,369","30,193","1,431","182,620","1,495","172,378"
1,경상남도,마산시,"326,663","171,791","31,539","122,814","14,066","1,476","169,895","1,896","154,872"
2,경상남도,진주시,"252,537","145,232","50,737","77,772","12,912","1,691","143,112","2,120","107,305"
3,경상남도,진해시,"118,111","70,223","15,379","46,643","6,344",751,"69,117","1,106","47,888"
4,경상남도,통영시,"102,469","60,728","13,684","41,786","3,423",827,"59,720","1,008","41,741"


In [177]:
gyeongnam_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               20 non-null     object
 1   구시군              20 non-null     object
 2   선거인수             20 non-null     object
 3   투표수              20 non-null     object
 4   득표수_1_열린우리당_김두관  20 non-null     object
 5   득표수_2_한나라당_김태호   20 non-null     object
 6   득표수_4_민주노동당_문성현  20 non-null     object
 7   득표수_5_국민중심당_김재주  20 non-null     object
 8   득표수_계            20 non-null     object
 9   무효투표수            20 non-null     object
 10  기권수              20 non-null     object
dtypes: object(11)
memory usage: 1.8+ KB


In [178]:
gyeongnam_4th = gyeongnam_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [179]:
# 수치형 열만 합계 구하기
summary_row = gyeongnam_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경상남도')

# summary_row를 맨 위에 붙이기
gyeongnam_4th_with_total = pd.concat([summary_row, gyeongnam_4th], ignore_index=True)

In [180]:
gyeongnam_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김두관,득표수_2_한나라당_김태호,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수
0,경상남도,합계,2375265,1373815,343137,852377,135823,18906,1350243,23572,1001450
1,경상남도,창원시,356493,184115,38627,112369,30193,1431,182620,1495,172378
2,경상남도,마산시,326663,171791,31539,122814,14066,1476,169895,1896,154872
3,경상남도,진주시,252537,145232,50737,77772,12912,1691,143112,2120,107305
4,경상남도,진해시,118111,70223,15379,46643,6344,751,69117,1106,47888
5,경상남도,통영시,102469,60728,13684,41786,3423,827,59720,1008,41741
6,경상남도,고성군,46167,33469,7661,21513,2686,754,32614,855,12698
7,경상남도,사천시,86136,57392,15408,35061,4718,962,56149,1243,28744
8,경상남도,김해시,315263,150966,47354,87632,12929,1213,149128,1838,164297
9,경상남도,밀양시,90362,58412,15879,36650,3844,825,57198,1214,31950


In [181]:
gyeongnam_4th_with_total.to_csv("temp1_governor_gyeongnam_4.csv", index=False, encoding="utf-8-sig")


## Jeju

In [182]:
jeju_4th = process_4th_governor_election_batch(
    user="sw1kwon",
    repo="korean-elections",
    folder="original/Local_Elections_Governor/4th_2006/49_제주"
)

4대 지방선거 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Local_Elections_Governor/4th_2006/49_제주
------------------------------------------------------------
폴더에서 4개 파일 발견

처리 중: 4901_제주시_제주도.xls (지역: 제주시)
WARNING *** file size (212874) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 29

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 29행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4902_북제주군_제주도.xls (지역: 북제주군)
WARNING *** file size (101411) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 10

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 10행 → 필터링 후: 1행
  ✓ 완료: 1행 추가

처리 중: 4903_서귀포시_제주도.xls (지역: 서귀포시)
WARNING *** file size (145824) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
헤더 행: 4행 ~ 6행
생성된 컬럼 수: 15
데이터 행 수: 21

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 21행 → 필터링 후: 1행
  ✓ 완료: 1

In [183]:
jeju_4th

,구시군명,column_0,읍면동명,선거인수,column_3,투표수,후보자별 득표수_열린우리당\n진철훈,한나라당\n현명관,무소속\n김태환,column_8,계,무효\n투표수,기권수,column_12,column_13,column_14
0,제주시,NaN,합계,"218,768",NaN,"138,717","22,748","56,423","58,722",NaN,"137,893",824,"80,051",NaN,NaN,0.0
1,북제주군,NaN,합계,"74,387",NaN,"51,650","10,793","16,607","23,455",NaN,"50,855",795,"22,737",NaN,NaN,0.0
2,서귀포시,NaN,합계,"62,209",NaN,"43,561","5,087","19,930","18,098",NaN,"43,115",446,"18,648",NaN,NaN,0.0
3,남제주군,NaN,합계,"56,498",NaN,"43,075","5,706","19,814","16,969",NaN,"42,489",586,"13,423",NaN,NaN,0.0


In [184]:
jeju_4th.columns.tolist()

['구시군명',
 'column_0',
 '읍면동명',
 '선거인수',
 'column_3',
 '투표수',
 '           후보자별 득표수_열린우리당\n진철훈',
 '한나라당\n현명관',
 '무소속\n김태환',
 'column_8',
 '계',
 '무효\n투표수',
 '기권수',
 'column_12',
 'column_13',
 'column_14']

In [185]:
rename_jeju = {
    '구시군명': '구시군',
    '           후보자별 득표수_열린우리당\n진철훈': '득표수_1_열린우리당_진철훈',
    '한나라당\n현명관': '득표수_2_한나라당_현명관',
    '무소속\n김태환': '득표수_6_무소속_김태환',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [186]:
jeju_4th = jeju_4th.rename(columns=rename_jeju).drop(columns=['column_0', '읍면동명', 'column_3', 'column_8', 'column_12', 'column_13', 'column_14'])
jeju_4th

,구시군,선거인수,투표수,득표수_1_열린우리당_진철훈,득표수_2_한나라당_현명관,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수
0,제주시,"218,768","138,717","22,748","56,423","58,722","137,893",824,"80,051"
1,북제주군,"74,387","51,650","10,793","16,607","23,455","50,855",795,"22,737"
2,서귀포시,"62,209","43,561","5,087","19,930","18,098","43,115",446,"18,648"
3,남제주군,"56,498","43,075","5,706","19,814","16,969","42,489",586,"13,423"


In [187]:
jeju_4th = jeju_4th.assign(
    시도='제주특별자치도'
)[['시도'] + jeju_4th.columns.tolist()]

In [188]:
jeju_4th.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진철훈,득표수_2_한나라당_현명관,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수
0,제주특별자치도,제주시,"218,768","138,717","22,748","56,423","58,722","137,893",824,"80,051"
1,제주특별자치도,북제주군,"74,387","51,650","10,793","16,607","23,455","50,855",795,"22,737"
2,제주특별자치도,서귀포시,"62,209","43,561","5,087","19,930","18,098","43,115",446,"18,648"
3,제주특별자치도,남제주군,"56,498","43,075","5,706","19,814","16,969","42,489",586,"13,423"


In [189]:
jeju_4th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   시도               4 non-null      object
 1   구시군              4 non-null      object
 2   선거인수             4 non-null      object
 3   투표수              4 non-null      object
 4   득표수_1_열린우리당_진철훈  4 non-null      object
 5   득표수_2_한나라당_현명관   4 non-null      object
 6   득표수_6_무소속_김태환    4 non-null      object
 7   득표수_계            4 non-null      object
 8   무효투표수            4 non-null      object
 9   기권수              4 non-null      object
dtypes: object(10)
memory usage: 452.0+ bytes


In [190]:
jeju_4th = jeju_4th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [191]:
# 수치형 열만 합계 구하기
summary_row = jeju_4th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '제주특별자치도')

# summary_row를 맨 위에 붙이기
jeju_4th_with_total = pd.concat([summary_row, jeju_4th], ignore_index=True)

In [192]:
jeju_4th_with_total

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진철훈,득표수_2_한나라당_현명관,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,411862,277003,44334,112774,117244,274352,2651,134859
1,제주특별자치도,제주시,218768,138717,22748,56423,58722,137893,824,80051
2,제주특별자치도,북제주군,74387,51650,10793,16607,23455,50855,795,22737
3,제주특별자치도,서귀포시,62209,43561,5087,19930,18098,43115,446,18648
4,제주특별자치도,남제주군,56498,43075,5706,19814,16969,42489,586,13423


In [193]:
jeju_4th_with_total.to_csv("temp1_governor_jeju_4.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [194]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp1_governor_jeonnam_4.csv
Added: temp1_governor_incheon_4.csv
Added: temp1_governor_jeju_4.csv
Added: temp1_governor_ulsan_4.csv
Added: temp1_governor_daejeon_4.csv
Added: temp1_governor_daegu_4.csv
Added: temp1_governor_gyeonggi_4.csv
Added: temp1_governor_seoul_4.csv
Added: temp1_governor_gyeongnam_4.csv
Added: temp1_governor_gyeongbuk_4.csv
Added: temp1_governor_chungnam_4.csv
Added: temp1_governor_jeonbuk_4.csv
Added: temp1_governor_chungbuk_4.csv
Added: temp1_governor_gwangju_4.csv
Added: temp1_governor_gangwon_4.csv
Added: temp1_governor_busan_4.csv
Total 16 files compressed.
